#### Primary Italian company active internationally in the retail of perfumery and cosmetics goods.
- Within the company, a dedicated unit is in charge of developing, planning and reporting of training activities towards Employee.
- In order to justify the investment in training activities the company asks for your support.

**HR_SALES:**
- **Retail Units:** Number of non accessory product sold in a given date
- **Commodity   :** Type of product sold (Accessory vs Item)
- **Sales       :** Local currency amount (currency is uniform within the dataset)
- **USER_ID     :** Employee's identifier

**HR_TRAINING**:
- **EmployeeType:** Coding related to the contract type
- **HRSCmpleted :** Hours of training activities completed
- **HIRING      :** Date of hiring (progressive)
- **Enrol       :** Type of enrolment for the specific training activity
- **USER_ID     :** Employee's identifier
- **CourseType  :** Coding of the course content type
- The training activities are available through smartphone app made up of short modules (5,10,20 mins etc) that can be completed during worktime.

**QUESTIONS**:
- Do training activities actually have an effect on performances?
- How is this effect quantifiable?
- Is the alleged effect heterogeneous along any observable dimension?

**KEY ASPECTS**:
- Accounting for possible confounding factors
- Choice of the proper granularity of the time dimension

In [1]:
import warnings
warnings.filterwarnings('ignore') # uncomment this line to suppress warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer

import datetime

sales= pd.read_csv('HR_sales_clean.csv')
sales


,Fiscal Quarter (SS),Fiscal Month (SS),Fiscal Week (SS),Retail Units,Date,Commodity,Sales,USER_ID
0,2018Q1,201802,201807,0.0,2018-02-16,ACCESSORIES,6.2608,9
1,2018Q1,201803,201809,0.0,2018-02-28,ACCESSORIES,10.0688,9
2,2018Q1,201803,201809,0.0,2018-03-02,ACCESSORIES,11.1888,9
3,2018Q1,201803,201810,0.0,2018-03-05,ACCESSORIES,10.0688,9
4,2018Q1,201803,201811,0.0,2018-03-12,ACCESSORIES,6.7088,9
...,...,...,...,...,...,...,...,...
8913289,2019Q4,201910,201942,0.0,2019-10-13,ACCESSORIES,49.2240,1000008
8913290,2019Q4,201910,201942,0.0,2019-10-14,ACCESSORIES,91.7280,1000008
8913291,2019Q4,201910,201942,1.0,2019-10-14,ITEM,77.6160,1000008
8913292,2019Q4,201910,201942,1.0,2019-10-14,ITEM,77.6160,1000008


In [2]:
sales.dtypes

Fiscal Quarter (SS)     object
Fiscal Month (SS)        int64
Fiscal Week (SS)         int64
Retail Units           float64
Date                    object
Commodity               object
Sales                  float64
USER_ID                  int64
dtype: object

In [3]:
train= pd.read_csv('HR_training_clean.csv')
train

,EmployeeType,HiringDate_HR,Hours Delivered,Enrol,Time Completed,USER_ID,CourseType
0,C,43693.0,00:05:00,Self,2019-10-10,747221,Corporate
1,C,43703.0,00:05:00,Self,2019-10-09,747960,Corporate
2,C,43710.0,00:05:00,Self,2019-09-27,747860,Corporate
3,C,43716.0,00:05:00,Self,2019-10-22,748365,Corporate
4,C,43723.0,00:05:00,Self,2019-10-11,977662,Corporate
...,...,...,...,...,...,...,...
408579,P_PT,42495.0,00:10:00,Self,2019-10-10,585421,Product
408580,P_PT,42634.0,00:10:00,Self,2019-09-21,513781,Product
408581,P_PT,43051.0,00:10:00,Self,2019-10-01,970908,Product
408582,P_PT,43457.0,00:10:00,Self,2019-07-15,737693,Product


In [4]:
#train['CourseType'].unique()
#train['EmployeeTime'].unique()
#train['HiringDate_HR'].unique()
#train['Enrol'].unique()
#np.unique(train['USER_ID'], return_counts = True)
train.USER_ID.nunique()     #value_counts()
#train['CourseType'].unique()

22161

In [5]:
train.dtypes

EmployeeType        object
HiringDate_HR      float64
Hours Delivered     object
Enrol               object
Time Completed      object
USER_ID              int64
CourseType          object
dtype: object

In [6]:
train['Hours Delivered'].unique()

array(['00:05:00', '00:10:00', '00:15:00', '00:20:00', '00:25:00',
       '00:30:00', '00:00:00', '01:30:00', '00:45:00', '00:04:00'],
      dtype=object)

In [7]:
sales['Commodity'].unique()

array(['ACCESSORIES', 'ITEM'], dtype=object)

In [8]:
train['Time Completed']

0         2019-10-10
1         2019-10-09
2         2019-09-27
3         2019-10-22
4         2019-10-11
             ...    
408579    2019-10-10
408580    2019-09-21
408581    2019-10-01
408582    2019-07-15
408583    2019-07-26
Name: Time Completed, Length: 408584, dtype: object

In [9]:
train['Time'] = pd.to_datetime(train['Time Completed'], format= "%Y-%m-%d")
train['Week'] = train['Time'].dt.week
train['Week'] = train['Week'].fillna(0)
train['Week'] = train['Week'].astype('int64')

In [10]:
train['Year'] = train['Time'].dt.year
train['Year'] = train['Year'].fillna(0)
train['Year'] = train['Year'].astype('int64')


In [11]:
train['Fiscal Week'] = train['Year']*100 + train['Week']
train['Fiscal Week'] 


0         201941
1         201941
2         201939
3         201943
4         201941
           ...  
408579    201941
408580    201938
408581    201940
408582    201929
408583    201930
Name: Fiscal Week, Length: 408584, dtype: int64

In [12]:
train['Hours Delivered2'] = pd.to_timedelta(train['Hours Delivered'])#, format="%H:%M:%S")


conditions = [
    train['Hours Delivered'] == '00:05:00',
    train['Hours Delivered'] == '00:10:00',
    train['Hours Delivered'] == '00:15:00',
    train['Hours Delivered'] == '00:20:00',
    train['Hours Delivered'] == '00:25:00',
    train['Hours Delivered'] == '00:30:00',
    train['Hours Delivered'] == '00:00:00',
    train['Hours Delivered'] == '01:30:00',
    train['Hours Delivered'] == '00:45:00',
    train['Hours Delivered'] == '00:04:00']
choices = [5,10,15,20,25,30,0,90,45,4]
train['Hours Delivered3'] = np.select(conditions, choices, default=4)
train


,EmployeeType,HiringDate_HR,Hours Delivered,Enrol,Time Completed,USER_ID,CourseType,Time,Week,Year,Fiscal Week,Hours Delivered2,Hours Delivered3
0,C,43693.0,00:05:00,Self,2019-10-10,747221,Corporate,2019-10-10,41,2019,201941,00:05:00,5
1,C,43703.0,00:05:00,Self,2019-10-09,747960,Corporate,2019-10-09,41,2019,201941,00:05:00,5
2,C,43710.0,00:05:00,Self,2019-09-27,747860,Corporate,2019-09-27,39,2019,201939,00:05:00,5
3,C,43716.0,00:05:00,Self,2019-10-22,748365,Corporate,2019-10-22,43,2019,201943,00:05:00,5
4,C,43723.0,00:05:00,Self,2019-10-11,977662,Corporate,2019-10-11,41,2019,201941,00:05:00,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
408579,P_PT,42495.0,00:10:00,Self,2019-10-10,585421,Product,2019-10-10,41,2019,201941,00:10:00,10
408580,P_PT,42634.0,00:10:00,Self,2019-09-21,513781,Product,2019-09-21,38,2019,201938,00:10:00,10
408581,P_PT,43051.0,00:10:00,Self,2019-10-01,970908,Product,2019-10-01,40,2019,201940,00:10:00,10
408582,P_PT,43457.0,00:10:00,Self,2019-07-15,737693,Product,2019-07-15,29,2019,201929,00:10:00,10


In [13]:
train['Time'].isna().unique()

array([False,  True])

In [14]:
train.groupby(['USER_ID','Time'])['Hours Delivered3'].sum().groupby(level=0).cumsum().reset_index()

,USER_ID,Time,Hours Delivered3
0,402,2018-07-10,5
1,402,2018-07-30,25
2,402,2018-08-13,60
3,402,2018-09-12,110
4,402,2018-09-13,120
...,...,...,...
240269,978708,2019-10-20,85
240270,978715,2019-10-20,25
240271,978723,2019-10-21,10
240272,978723,2019-10-22,30


In [15]:
train.groupby(['USER_ID','Fiscal Week'])['Hours Delivered3'].sum().groupby(level=0).cumsum().reset_index()

,USER_ID,Fiscal Week,Hours Delivered3
0,402,201828,5
1,402,201831,25
2,402,201833,60
3,402,201837,120
4,402,201839,160
...,...,...,...
202282,978732,0,0
202283,978733,0,0
202284,978734,0,0
202285,978735,0,0


In [16]:
unique = train['USER_ID'].unique()
sales.loc[sales['USER_ID'].isin(unique)]

,Fiscal Quarter (SS),Fiscal Month (SS),Fiscal Week (SS),Retail Units,Date,Commodity,Sales,USER_ID
31,2018Q1,201801,201801,1.0,2018-01-02,ITEM,55.9888,1035
32,2018Q1,201801,201801,1.0,2018-01-02,ITEM,145.5888,1035
33,2018Q1,201801,201801,1.0,2018-01-04,ITEM,55.9888,1035
34,2018Q1,201801,201801,1.0,2018-01-05,ITEM,167.9440,1035
35,2018Q1,201801,201801,1.0,2018-01-03,ITEM,55.9888,1035
...,...,...,...,...,...,...,...,...
8585663,2019Q4,201910,201942,1.0,2019-10-14,ITEM,364.0000,978685
8585664,2019Q4,201910,201942,1.0,2019-10-14,ITEM,229.6000,978685
8585665,2019Q4,201910,201942,1.0,2019-10-15,ITEM,238.5600,978686
8585666,2019Q4,201910,201942,1.0,2019-10-13,ITEM,213.9200,978686


In [17]:
sales['Date'] = pd.to_datetime(sales['Date'], format= "%Y-%m-%d")

    

In [18]:
#for user in sorted(train['USER_ID'].unique()):
    #df = train[train['USER_ID']== user]
    #df.groupby('USER_ID').sum()
    #for i in df.index:
        #timeList = df['Hours Delivered'][i]
        #print(i)
        #for tm in timeList:
            #print(tm)
        #totalSecs = 0
        #for tm in timeList:
            #timeParts = [int(s) for s in tm.split(':')]
            #totalSecs += (timeParts[0] * 60 + timeParts[1]) * 60 + timeParts[2]
        #totalSecs, sec = divmod(totalSecs, 60)
        #hr, min = divmod(totalSecs, 60)
        #print("%d:%02d:%02d" % (hr, min, sec))

In [19]:
HoursToMinutes = train['Hours Delivered'].str.split(':').str[0]
HoursToMinutes= HoursToMinutes.astype(int)*60

Minutes=train['Hours Delivered'].str.split(':').str[1]
Minutes= Minutes.astype(int)

train['TotalMinutesDelivered'] = HoursToMinutes+Minutes

# Removing null values of 'HiringDate_HR' and 0 values of 'TotalMinutesDelivered'
train.drop(train[train['HiringDate_HR'].isnull()][train['TotalMinutesDelivered']==0].index, inplace=True)

# Removing null values of 'Time Completed' and 0 values of 'TotalMinutesDelivered'
train.drop(train[train['Time Completed'].isnull()][train['TotalMinutesDelivered']==0].index, inplace=True)

train

,EmployeeType,HiringDate_HR,Hours Delivered,Enrol,Time Completed,USER_ID,CourseType,Time,Week,Year,Fiscal Week,Hours Delivered2,Hours Delivered3,TotalMinutesDelivered
0,C,43693.0,00:05:00,Self,2019-10-10,747221,Corporate,2019-10-10,41,2019,201941,00:05:00,5,5
1,C,43703.0,00:05:00,Self,2019-10-09,747960,Corporate,2019-10-09,41,2019,201941,00:05:00,5,5
2,C,43710.0,00:05:00,Self,2019-09-27,747860,Corporate,2019-09-27,39,2019,201939,00:05:00,5,5
3,C,43716.0,00:05:00,Self,2019-10-22,748365,Corporate,2019-10-22,43,2019,201943,00:05:00,5,5
4,C,43723.0,00:05:00,Self,2019-10-11,977662,Corporate,2019-10-11,41,2019,201941,00:05:00,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
408579,P_PT,42495.0,00:10:00,Self,2019-10-10,585421,Product,2019-10-10,41,2019,201941,00:10:00,10,10
408580,P_PT,42634.0,00:10:00,Self,2019-09-21,513781,Product,2019-09-21,38,2019,201938,00:10:00,10,10
408581,P_PT,43051.0,00:10:00,Self,2019-10-01,970908,Product,2019-10-01,40,2019,201940,00:10:00,10,10
408582,P_PT,43457.0,00:10:00,Self,2019-07-15,737693,Product,2019-07-15,29,2019,201929,00:10:00,10,10


In [20]:
# Convert 'HiringDate_HR' format to Date format yyyy-mm-dd
def convert_excel_time(excel_time):

    return pd.to_datetime('1900-01-01') + pd.to_timedelta(excel_time,'D')

train['HiringDate_HR'] = convert_excel_time(train['HiringDate_HR']-2)
train['HiringDate_HR'] = train['HiringDate_HR'].dt.strftime('%Y-%m-%d')

train

,EmployeeType,HiringDate_HR,Hours Delivered,Enrol,Time Completed,USER_ID,CourseType,Time,Week,Year,Fiscal Week,Hours Delivered2,Hours Delivered3,TotalMinutesDelivered
0,C,2019-08-16,00:05:00,Self,2019-10-10,747221,Corporate,2019-10-10,41,2019,201941,00:05:00,5,5
1,C,2019-08-26,00:05:00,Self,2019-10-09,747960,Corporate,2019-10-09,41,2019,201941,00:05:00,5,5
2,C,2019-09-02,00:05:00,Self,2019-09-27,747860,Corporate,2019-09-27,39,2019,201939,00:05:00,5,5
3,C,2019-09-08,00:05:00,Self,2019-10-22,748365,Corporate,2019-10-22,43,2019,201943,00:05:00,5,5
4,C,2019-09-15,00:05:00,Self,2019-10-11,977662,Corporate,2019-10-11,41,2019,201941,00:05:00,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
408579,P_PT,2016-05-05,00:10:00,Self,2019-10-10,585421,Product,2019-10-10,41,2019,201941,00:10:00,10,10
408580,P_PT,2016-09-21,00:10:00,Self,2019-09-21,513781,Product,2019-09-21,38,2019,201938,00:10:00,10,10
408581,P_PT,2017-11-12,00:10:00,Self,2019-10-01,970908,Product,2019-10-01,40,2019,201940,00:10:00,10,10
408582,P_PT,2018-12-23,00:10:00,Self,2019-07-15,737693,Product,2019-07-15,29,2019,201929,00:10:00,10,10


In [21]:
# Correcting EmployeeType Value in train with mode value
train['EmployeeType'] = train['EmployeeType'].replace(['-'], [train['EmployeeType'].mode()])
train

,EmployeeType,HiringDate_HR,Hours Delivered,Enrol,Time Completed,USER_ID,CourseType,Time,Week,Year,Fiscal Week,Hours Delivered2,Hours Delivered3,TotalMinutesDelivered
0,C,2019-08-16,00:05:00,Self,2019-10-10,747221,Corporate,2019-10-10,41,2019,201941,00:05:00,5,5
1,C,2019-08-26,00:05:00,Self,2019-10-09,747960,Corporate,2019-10-09,41,2019,201941,00:05:00,5,5
2,C,2019-09-02,00:05:00,Self,2019-09-27,747860,Corporate,2019-09-27,39,2019,201939,00:05:00,5,5
3,C,2019-09-08,00:05:00,Self,2019-10-22,748365,Corporate,2019-10-22,43,2019,201943,00:05:00,5,5
4,C,2019-09-15,00:05:00,Self,2019-10-11,977662,Corporate,2019-10-11,41,2019,201941,00:05:00,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
408579,P_PT,2016-05-05,00:10:00,Self,2019-10-10,585421,Product,2019-10-10,41,2019,201941,00:10:00,10,10
408580,P_PT,2016-09-21,00:10:00,Self,2019-09-21,513781,Product,2019-09-21,38,2019,201938,00:10:00,10,10
408581,P_PT,2017-11-12,00:10:00,Self,2019-10-01,970908,Product,2019-10-01,40,2019,201940,00:10:00,10,10
408582,P_PT,2018-12-23,00:10:00,Self,2019-07-15,737693,Product,2019-07-15,29,2019,201929,00:10:00,10,10


In [22]:
train.isnull().sum()

EmployeeType             0
HiringDate_HR            0
Hours Delivered          0
Enrol                    0
Time Completed           0
USER_ID                  0
CourseType               0
Time                     0
Week                     0
Year                     0
Fiscal Week              0
Hours Delivered2         0
Hours Delivered3         0
TotalMinutesDelivered    0
dtype: int64

In [23]:
train['USER_ID'] = train['USER_ID'].astype(object)
train['Enrol'] = train['Enrol'].replace('-','Mandatory')

# Creating Dummy Variables of Course Type
train2=train.groupby(['Time Completed','USER_ID','EmployeeType','HiringDate_HR','CourseType','Enrol'])['TotalMinutesDelivered']\
.sum().reset_index()

train2=pd.get_dummies(train2, columns=['CourseType'])
train2=pd.get_dummies(train2, columns=['Enrol'])

train2['CourseType_Corporate_Self']=train2['CourseType_Corporate']*train2['TotalMinutesDelivered']*train2['Enrol_Self']
train2['CourseType_Product_Self']=train2['CourseType_Product']*train2['TotalMinutesDelivered']*train2['Enrol_Self']
train2['CourseType_Theme_Self']=train2['CourseType_Theme']*train2['TotalMinutesDelivered']*train2['Enrol_Self']
train2['CourseType_Corporate_Mandatory']=train2['CourseType_Corporate']*train2['TotalMinutesDelivered']*train2['Enrol_Mandatory']
train2['CourseType_Product_Mandatory']=train2['CourseType_Product']*train2['TotalMinutesDelivered']*train2['Enrol_Mandatory']
train2['CourseType_Theme_Mandatory']=train2['CourseType_Theme']*train2['TotalMinutesDelivered']*train2['Enrol_Mandatory']


In [24]:
train2

,Time Completed,USER_ID,EmployeeType,HiringDate_HR,TotalMinutesDelivered,CourseType_Corporate,CourseType_Product,CourseType_Theme,Enrol_Mandatory,Enrol_Self,CourseType_Corporate_Self,CourseType_Product_Self,CourseType_Theme_Self,CourseType_Corporate_Mandatory,CourseType_Product_Mandatory,CourseType_Theme_Mandatory
0,2017-08-11,813113,P_FT,2015-11-09,10,1,0,0,0,1,10,0,0,0,0,0
1,2017-08-16,592129,P_FT,2016-09-06,10,1,0,0,0,1,10,0,0,0,0,0
2,2017-08-20,812045,P_FT,2015-07-20,10,1,0,0,0,1,10,0,0,0,0,0
3,2017-08-28,955849,P_FT,2019-06-21,10,1,0,0,0,1,10,0,0,0,0,0
4,2017-09-07,812447,P_PT,2015-07-22,10,1,0,0,0,1,10,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301176,2019-10-23,978485,C,2019-08-25,25,1,0,0,0,1,25,0,0,0,0,0
301177,2019-10-23,978555,P_FT,2019-09-15,10,0,0,1,0,1,0,0,10,0,0,0
301178,2019-10-23,978643,C,2019-10-08,5,1,0,0,0,1,5,0,0,0,0,0
301179,2019-10-23,978643,C,2019-10-08,20,0,1,0,1,0,0,0,0,0,20,0


In [25]:
train3=train2.groupby(['Time Completed','USER_ID','EmployeeType','HiringDate_HR'])['CourseType_Corporate_Self','CourseType_Product_Self','CourseType_Theme_Self','CourseType_Corporate_Mandatory','CourseType_Product_Mandatory','CourseType_Theme_Mandatory'].sum().reset_index()
train3

,Time Completed,USER_ID,EmployeeType,HiringDate_HR,CourseType_Corporate_Self,CourseType_Product_Self,CourseType_Theme_Self,CourseType_Corporate_Mandatory,CourseType_Product_Mandatory,CourseType_Theme_Mandatory
0,2017-08-11,813113,P_FT,2015-11-09,10,0,0,0,0,0
1,2017-08-16,592129,P_FT,2016-09-06,10,0,0,0,0,0
2,2017-08-20,812045,P_FT,2015-07-20,10,0,0,0,0,0
3,2017-08-28,955849,P_FT,2019-06-21,10,0,0,0,0,0
4,2017-09-07,812447,P_PT,2015-07-22,10,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
240269,2019-10-23,976654,P_FT,2018-03-09,15,0,0,0,0,0
240270,2019-10-23,978053,P_FT,2019-05-10,15,0,0,0,0,0
240271,2019-10-23,978485,C,2019-08-25,25,0,0,5,0,0
240272,2019-10-23,978555,P_FT,2019-09-15,0,0,10,0,0,0


In [26]:
train3[train3['USER_ID']==592129].sort_values(by='Time Completed')

,Time Completed,USER_ID,EmployeeType,HiringDate_HR,CourseType_Corporate_Self,CourseType_Product_Self,CourseType_Theme_Self,CourseType_Corporate_Mandatory,CourseType_Product_Mandatory,CourseType_Theme_Mandatory
1,2017-08-16,592129,P_FT,2016-09-06,10,0,0,0,0,0
10,2017-11-09,592129,P_FT,2016-09-06,10,0,0,0,0,0
26,2018-02-14,592129,P_FT,2016-09-06,10,0,0,0,0,0
36,2018-03-02,592129,P_FT,2016-09-06,10,0,0,0,0,0
42,2018-03-21,592129,P_FT,2016-09-06,0,10,0,0,0,0
50,2018-04-04,592129,P_FT,2016-09-06,0,10,0,0,0,0
9896,2018-07-16,592129,P_FT,2016-09-06,0,10,0,0,0,0
88747,2018-12-17,592129,P_FT,2016-09-06,30,0,0,0,0,0
136406,2019-04-02,592129,P_FT,2016-09-06,10,20,0,0,0,0
148997,2019-04-29,592129,P_FT,2016-09-06,5,0,0,0,0,0


In [27]:
train3=train2.groupby(['Time Completed','USER_ID','EmployeeType','HiringDate_HR'])['CourseType_Corporate_Self',\
             'CourseType_Product_Self','CourseType_Theme_Self','CourseType_Corporate_Mandatory','CourseType_Product_Mandatory',\
             'CourseType_Theme_Mandatory'].sum().groupby('USER_ID').cumsum().reset_index()

train3=train3.sort_values(by=['USER_ID','Time Completed'])
train3

,Time Completed,USER_ID,EmployeeType,HiringDate_HR,CourseType_Corporate_Self,CourseType_Product_Self,CourseType_Theme_Self,CourseType_Corporate_Mandatory,CourseType_Product_Mandatory,CourseType_Theme_Mandatory
8427,2018-07-10,402,P_FT,1985-02-07,5,0,0,0,0,0
20850,2018-07-30,402,P_FT,1985-02-07,5,0,0,0,20,0
28099,2018-08-13,402,P_FT,1985-02-07,5,0,10,0,45,0
43351,2018-09-12,402,P_FT,1985-02-07,5,10,50,0,45,0
43853,2018-09-13,402,P_FT,1985-02-07,5,20,50,0,45,0
...,...,...,...,...,...,...,...,...,...,...
239144,2019-10-20,978708,C,2019-10-17,45,10,30,0,0,0
239145,2019-10-20,978715,C,2019-10-17,20,0,0,5,0,0
239438,2019-10-21,978723,C,2019-10-21,0,0,10,0,0,0
240121,2019-10-22,978723,C,2019-10-21,15,0,10,5,0,0


In [28]:
train3[train3['USER_ID']==402].sort_values(by='Time Completed')  #592129

,Time Completed,USER_ID,EmployeeType,HiringDate_HR,CourseType_Corporate_Self,CourseType_Product_Self,CourseType_Theme_Self,CourseType_Corporate_Mandatory,CourseType_Product_Mandatory,CourseType_Theme_Mandatory
8427,2018-07-10,402,P_FT,1985-02-07,5,0,0,0,0,0
20850,2018-07-30,402,P_FT,1985-02-07,5,0,0,0,20,0
28099,2018-08-13,402,P_FT,1985-02-07,5,0,10,0,45,0
43351,2018-09-12,402,P_FT,1985-02-07,5,10,50,0,45,0
43853,2018-09-13,402,P_FT,1985-02-07,5,20,50,0,45,0
49503,2018-09-24,402,P_FT,1985-02-07,5,20,70,0,45,0
50169,2018-09-25,402,P_FT,1985-02-07,5,20,80,0,45,0
51927,2018-09-28,402,P_FT,1985-02-07,5,20,80,0,55,0
59729,2018-10-05,402,P_FT,1985-02-07,5,20,80,0,85,0
77639,2018-11-05,402,P_FT,1985-02-07,5,30,80,0,85,0


In [29]:
train3.dtypes

Time Completed                    object
USER_ID                            int64
EmployeeType                      object
HiringDate_HR                     object
CourseType_Corporate_Self          int32
CourseType_Product_Self            int32
CourseType_Theme_Self              int32
CourseType_Corporate_Mandatory     int32
CourseType_Product_Mandatory       int32
CourseType_Theme_Mandatory         int32
dtype: object

In [30]:
train3['Time Completed'] = pd.to_datetime(train3['Time Completed'], format= "%Y-%m-%d")
train3.dtypes

Time Completed                    datetime64[ns]
USER_ID                                    int64
EmployeeType                              object
HiringDate_HR                             object
CourseType_Corporate_Self                  int32
CourseType_Product_Self                    int32
CourseType_Theme_Self                      int32
CourseType_Corporate_Mandatory             int32
CourseType_Product_Mandatory               int32
CourseType_Theme_Mandatory                 int32
dtype: object

In [31]:
sales.dtypes

Fiscal Quarter (SS)            object
Fiscal Month (SS)               int64
Fiscal Week (SS)                int64
Retail Units                  float64
Date                   datetime64[ns]
Commodity                      object
Sales                         float64
USER_ID                         int64
dtype: object

In [32]:
sales = sales.rename(columns={'Date': 'Time Completed'})
sales = sales.sort_values(by=['USER_ID','Time Completed'])
sales

,Fiscal Quarter (SS),Fiscal Month (SS),Fiscal Week (SS),Retail Units,Time Completed,Commodity,Sales,USER_ID
0,2018Q1,201802,201807,0.0,2018-02-16,ACCESSORIES,6.2608,9
1,2018Q1,201803,201809,0.0,2018-02-28,ACCESSORIES,10.0688,9
2,2018Q1,201803,201809,0.0,2018-03-02,ACCESSORIES,11.1888,9
3,2018Q1,201803,201810,0.0,2018-03-05,ACCESSORIES,10.0688,9
4,2018Q1,201803,201811,0.0,2018-03-12,ACCESSORIES,6.7088,9
...,...,...,...,...,...,...,...,...
8913286,2019Q4,201910,201942,0.0,2019-10-14,ACCESSORIES,49.2240,1000008
8913287,2019Q4,201910,201942,0.0,2019-10-14,ACCESSORIES,91.7280,1000008
8913290,2019Q4,201910,201942,0.0,2019-10-14,ACCESSORIES,91.7280,1000008
8913291,2019Q4,201910,201942,1.0,2019-10-14,ITEM,77.6160,1000008


In [33]:
merge = pd.merge_asof(sales.sort_values(by=['Time Completed']), train3.sort_values(by=['Time Completed']),
                      on=['Time Completed'],
                      by='USER_ID')
merge

,Fiscal Quarter (SS),Fiscal Month (SS),Fiscal Week (SS),Retail Units,Time Completed,Commodity,Sales,USER_ID,EmployeeType,HiringDate_HR,CourseType_Corporate_Self,CourseType_Product_Self,CourseType_Theme_Self,CourseType_Corporate_Mandatory,CourseType_Product_Mandatory,CourseType_Theme_Mandatory
0,2018Q1,201801,201801,2.0,2018-01-01,ITEM,269.6400,588104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018Q1,201801,201801,0.0,2018-01-01,ACCESSORIES,11.8832,577781,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018Q1,201801,201801,2.0,2018-01-01,ITEM,329.7168,965271,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018Q1,201801,201801,1.0,2018-01-01,ITEM,71.6688,965271,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018Q1,201801,201801,1.0,2018-01-01,ITEM,195.3280,965271,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8913289,2019Q4,201910,201942,1.0,2019-10-15,ITEM,263.2000,511926,P_FT,2011-10-09,30.0,90.0,170.0,0.0,85.0,20.0
8913290,2019Q4,201910,201942,0.0,2019-10-15,ACCESSORIES,17.3488,511926,P_FT,2011-10-09,30.0,90.0,170.0,0.0,85.0,20.0
8913291,2019Q4,201910,201942,1.0,2019-10-15,ITEM,330.4000,511926,P_FT,2011-10-09,30.0,90.0,170.0,0.0,85.0,20.0
8913292,2019Q4,201910,201942,1.0,2019-10-15,ITEM,111.9440,511926,P_FT,2011-10-09,30.0,90.0,170.0,0.0,85.0,20.0


In [34]:
merge[merge['USER_ID']==511926].sort_values(by='Time Completed').head(500) 

#check tail(50 or 100) and see that on time 
#completed in merge the order of expertise is correct


,Fiscal Quarter (SS),Fiscal Month (SS),Fiscal Week (SS),Retail Units,Time Completed,Commodity,Sales,USER_ID,EmployeeType,HiringDate_HR,CourseType_Corporate_Self,CourseType_Product_Self,CourseType_Theme_Self,CourseType_Corporate_Mandatory,CourseType_Product_Mandatory,CourseType_Theme_Mandatory
3101,2018Q1,201801,201801,1.0,2018-01-01,ITEM,160.1600,511926,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3111,2018Q1,201801,201801,1.0,2018-01-01,ITEM,159.1520,511926,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44996,2018Q1,201801,201801,2.0,2018-01-05,ITEM,470.3888,511926,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44997,2018Q1,201801,201801,1.0,2018-01-05,ITEM,216.1600,511926,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44998,2018Q1,201801,201801,1.0,2018-01-05,ITEM,111.9888,511926,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3044072,2018Q3,201807,201830,1.0,2018-07-22,ITEM,167.9440,511926,P_FT,2011-10-09,15.0,10.0,0.0,0.0,15.0,0.0
3043455,2018Q3,201807,201830,1.0,2018-07-22,ITEM,110.8800,511926,P_FT,2011-10-09,15.0,10.0,0.0,0.0,15.0,0.0
3043428,2018Q3,201807,201830,1.0,2018-07-22,ITEM,274.4000,511926,P_FT,2011-10-09,15.0,10.0,0.0,0.0,15.0,0.0
3043403,2018Q3,201807,201830,1.0,2018-07-22,ITEM,134.3440,511926,P_FT,2011-10-09,15.0,10.0,0.0,0.0,15.0,0.0


In [35]:
train3[train3['USER_ID']==511926].sort_values(by='Time Completed')

,Time Completed,USER_ID,EmployeeType,HiringDate_HR,CourseType_Corporate_Self,CourseType_Product_Self,CourseType_Theme_Self,CourseType_Corporate_Mandatory,CourseType_Product_Mandatory,CourseType_Theme_Mandatory
853,2018-05-20,511926,P_FT,2011-10-09,5,0,0,0,0,0
2317,2018-06-02,511926,P_FT,2011-10-09,5,10,0,0,0,0
4732,2018-06-20,511926,P_FT,2011-10-09,10,10,0,0,0,0
10400,2018-07-17,511926,P_FT,2011-10-09,15,10,0,0,15,0
16758,2018-07-27,511926,P_FT,2011-10-09,15,10,0,0,35,0
24457,2018-08-04,511926,P_FT,2011-10-09,15,10,0,0,45,0
27871,2018-08-12,511926,P_FT,2011-10-09,15,10,10,0,45,0
32678,2018-08-28,511926,P_FT,2011-10-09,15,20,30,0,45,0
37430,2018-09-02,511926,P_FT,2011-10-09,15,20,50,0,45,0
50952,2018-09-26,511926,P_FT,2011-10-09,15,20,80,0,45,0


In [36]:
sales[sales['USER_ID']== 402].sort_values(by='Time Completed')

,Fiscal Quarter (SS),Fiscal Month (SS),Fiscal Week (SS),Retail Units,Time Completed,Commodity,Sales,USER_ID


In [37]:
missing= sorted(list(set(train3['USER_ID'].unique()) - set(merge['USER_ID'].unique())))

In [38]:
nosales = train3[train3['USER_ID'].isin(missing)].reset_index(drop=True)
nosales

,Time Completed,USER_ID,EmployeeType,HiringDate_HR,CourseType_Corporate_Self,CourseType_Product_Self,CourseType_Theme_Self,CourseType_Corporate_Mandatory,CourseType_Product_Mandatory,CourseType_Theme_Mandatory
0,2018-07-10,402,P_FT,1985-02-07,5,0,0,0,0,0
1,2018-07-30,402,P_FT,1985-02-07,5,0,0,0,20,0
2,2018-08-13,402,P_FT,1985-02-07,5,0,10,0,45,0
3,2018-09-12,402,P_FT,1985-02-07,5,10,50,0,45,0
4,2018-09-13,402,P_FT,1985-02-07,5,20,50,0,45,0
...,...,...,...,...,...,...,...,...,...,...
4934,2019-10-20,978708,C,2019-10-17,45,10,30,0,0,0
4935,2019-10-20,978715,C,2019-10-17,20,0,0,5,0,0
4936,2019-10-21,978723,C,2019-10-21,0,0,10,0,0,0
4937,2019-10-22,978723,C,2019-10-21,15,0,10,5,0,0


In [39]:
merge.append(nosales, ignore_index= True)

,Commodity,CourseType_Corporate_Mandatory,CourseType_Corporate_Self,CourseType_Product_Mandatory,CourseType_Product_Self,CourseType_Theme_Mandatory,CourseType_Theme_Self,EmployeeType,Fiscal Month (SS),Fiscal Quarter (SS),Fiscal Week (SS),HiringDate_HR,Retail Units,Sales,Time Completed,USER_ID
0,ITEM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,201801.0,2018Q1,201801.0,NaN,2.0,269.6400,2018-01-01,588104
1,ACCESSORIES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,201801.0,2018Q1,201801.0,NaN,0.0,11.8832,2018-01-01,577781
2,ITEM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,201801.0,2018Q1,201801.0,NaN,2.0,329.7168,2018-01-01,965271
3,ITEM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,201801.0,2018Q1,201801.0,NaN,1.0,71.6688,2018-01-01,965271
4,ITEM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,201801.0,2018Q1,201801.0,NaN,1.0,195.3280,2018-01-01,965271
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8918228,NaN,0.0,45.0,0.0,10.0,0.0,30.0,C,NaN,NaN,NaN,2019-10-17,NaN,NaN,2019-10-20,978708
8918229,NaN,5.0,20.0,0.0,0.0,0.0,0.0,C,NaN,NaN,NaN,2019-10-17,NaN,NaN,2019-10-20,978715
8918230,NaN,0.0,0.0,0.0,0.0,0.0,10.0,C,NaN,NaN,NaN,2019-10-21,NaN,NaN,2019-10-21,978723
8918231,NaN,5.0,15.0,0.0,0.0,0.0,10.0,C,NaN,NaN,NaN,2019-10-21,NaN,NaN,2019-10-22,978723


In [40]:
merge.isnull().sum()

Fiscal Quarter (SS)                     0
Fiscal Month (SS)                       0
Fiscal Week (SS)                        0
Retail Units                            0
Time Completed                          0
Commodity                               0
Sales                                   0
USER_ID                                 0
EmployeeType                      3226742
HiringDate_HR                     3226742
CourseType_Corporate_Self         3226742
CourseType_Product_Self           3226742
CourseType_Theme_Self             3226742
CourseType_Corporate_Mandatory    3226742
CourseType_Product_Mandatory      3226742
CourseType_Theme_Mandatory        3226742
dtype: int64

In [41]:
missing2= sorted(list(set(merge['USER_ID'].unique()) - set(train3['USER_ID'].unique())))
missing2 #those who sold something without any train at all

[9,
 10,
 11,
 5129,
 17989,
 34290,
 37129,
 41856,
 45119,
 45407,
 52780,
 58134,
 61752,
 61947,
 62080,
 62230,
 62993,
 70795,
 72797,
 73124,
 73426,
 73627,
 74446,
 78477,
 79881,
 81356,
 82011,
 86294,
 96456,
 99196,
 99421,
 100701,
 102643,
 104399,
 104993,
 115660,
 154054,
 161963,
 216523,
 263119,
 272440,
 273209,
 273954,
 275788,
 278049,
 279727,
 280370,
 280781,
 281456,
 281458,
 287946,
 288106,
 289169,
 289998,
 290027,
 291358,
 291998,
 300141,
 300189,
 300237,
 300351,
 300400,
 300464,
 301431,
 302988,
 303474,
 306550,
 311271,
 317233,
 318046,
 318095,
 319677,
 321451,
 326232,
 344931,
 356186,
 357851,
 365033,
 365227,
 365790,
 366704,
 390370,
 394868,
 400435,
 400436,
 400437,
 402775,
 406464,
 406765,
 406891,
 408000,
 410317,
 410489,
 413200,
 413331,
 415834,
 416874,
 418070,
 420863,
 424244,
 425788,
 426901,
 428519,
 428959,
 430509,
 430524,
 432572,
 439662,
 445973,
 447455,
 449148,
 452779,
 452907,
 453084,
 457220,
 457755

In [42]:
merge[merge['USER_ID'].isin(missing2)].reset_index(drop=True)

,Fiscal Quarter (SS),Fiscal Month (SS),Fiscal Week (SS),Retail Units,Time Completed,Commodity,Sales,USER_ID,EmployeeType,HiringDate_HR,CourseType_Corporate_Self,CourseType_Product_Self,CourseType_Theme_Self,CourseType_Corporate_Mandatory,CourseType_Product_Mandatory,CourseType_Theme_Mandatory
0,2018Q1,201801,201801,2.0,2018-01-01,ITEM,269.6400,588104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018Q1,201801,201801,1.0,2018-01-01,ITEM,182.5600,592146,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018Q1,201801,201801,1.0,2018-01-01,ITEM,111.9888,592146,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018Q1,201801,201801,1.0,2018-01-01,ITEM,111.9888,592146,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018Q1,201801,201801,1.0,2018-01-01,ITEM,55.9888,592146,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799256,2019Q4,201910,201942,1.0,2019-10-15,ITEM,204.9600,737066,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
799257,2019Q4,201910,201942,0.0,2019-10-15,ACCESSORIES,24.6176,737061,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
799258,2019Q4,201910,201942,0.0,2019-10-15,ACCESSORIES,11.1888,737065,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
799259,2019Q4,201910,201942,1.0,2019-10-15,ITEM,171.3600,586217,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
merge['CourseType_Corporate_Self'] = merge['CourseType_Corporate_Self'].fillna(0)
merge['CourseType_Product_Self'] = merge['CourseType_Product_Self'].fillna(0)
merge['CourseType_Theme_Self'] = merge['CourseType_Theme_Self'].fillna(0)
merge['CourseType_Corporate_Mandatory'] = merge['CourseType_Corporate_Mandatory'].fillna(0)
merge['CourseType_Product_Mandatory'] = merge['CourseType_Product_Mandatory'].fillna(0)
merge['CourseType_Theme_Mandatory'] = merge['CourseType_Theme_Mandatory'].fillna(0)
merge #filled with 0 because they were both those who never got any train and those who still haven't got train yet at that moment

,Fiscal Quarter (SS),Fiscal Month (SS),Fiscal Week (SS),Retail Units,Time Completed,Commodity,Sales,USER_ID,EmployeeType,HiringDate_HR,CourseType_Corporate_Self,CourseType_Product_Self,CourseType_Theme_Self,CourseType_Corporate_Mandatory,CourseType_Product_Mandatory,CourseType_Theme_Mandatory
0,2018Q1,201801,201801,2.0,2018-01-01,ITEM,269.6400,588104,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
1,2018Q1,201801,201801,0.0,2018-01-01,ACCESSORIES,11.8832,577781,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
2,2018Q1,201801,201801,2.0,2018-01-01,ITEM,329.7168,965271,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
3,2018Q1,201801,201801,1.0,2018-01-01,ITEM,71.6688,965271,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
4,2018Q1,201801,201801,1.0,2018-01-01,ITEM,195.3280,965271,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8913289,2019Q4,201910,201942,1.0,2019-10-15,ITEM,263.2000,511926,P_FT,2011-10-09,30.0,90.0,170.0,0.0,85.0,20.0
8913290,2019Q4,201910,201942,0.0,2019-10-15,ACCESSORIES,17.3488,511926,P_FT,2011-10-09,30.0,90.0,170.0,0.0,85.0,20.0
8913291,2019Q4,201910,201942,1.0,2019-10-15,ITEM,330.4000,511926,P_FT,2011-10-09,30.0,90.0,170.0,0.0,85.0,20.0
8913292,2019Q4,201910,201942,1.0,2019-10-15,ITEM,111.9440,511926,P_FT,2011-10-09,30.0,90.0,170.0,0.0,85.0,20.0


In [44]:
mergenot= merge[~merge['USER_ID'].isin(missing2)].reset_index(drop=True)
mergenot #only individuals that have a train at a certain type, this can be filled in every attribute

,Fiscal Quarter (SS),Fiscal Month (SS),Fiscal Week (SS),Retail Units,Time Completed,Commodity,Sales,USER_ID,EmployeeType,HiringDate_HR,CourseType_Corporate_Self,CourseType_Product_Self,CourseType_Theme_Self,CourseType_Corporate_Mandatory,CourseType_Product_Mandatory,CourseType_Theme_Mandatory
0,2018Q1,201801,201801,0.0,2018-01-01,ACCESSORIES,11.8832,577781,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
1,2018Q1,201801,201801,2.0,2018-01-01,ITEM,329.7168,965271,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
2,2018Q1,201801,201801,1.0,2018-01-01,ITEM,71.6688,965271,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
3,2018Q1,201801,201801,1.0,2018-01-01,ITEM,195.3280,965271,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
4,2018Q1,201801,201801,1.0,2018-01-01,ITEM,216.1600,965271,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8114028,2019Q4,201910,201942,1.0,2019-10-15,ITEM,263.2000,511926,P_FT,2011-10-09,30.0,90.0,170.0,0.0,85.0,20.0
8114029,2019Q4,201910,201942,0.0,2019-10-15,ACCESSORIES,17.3488,511926,P_FT,2011-10-09,30.0,90.0,170.0,0.0,85.0,20.0
8114030,2019Q4,201910,201942,1.0,2019-10-15,ITEM,330.4000,511926,P_FT,2011-10-09,30.0,90.0,170.0,0.0,85.0,20.0
8114031,2019Q4,201910,201942,1.0,2019-10-15,ITEM,111.9440,511926,P_FT,2011-10-09,30.0,90.0,170.0,0.0,85.0,20.0


In [45]:
mergenot[mergenot['USER_ID']== 577781].sort_values(by='Time Completed')

,Fiscal Quarter (SS),Fiscal Month (SS),Fiscal Week (SS),Retail Units,Time Completed,Commodity,Sales,USER_ID,EmployeeType,HiringDate_HR,CourseType_Corporate_Self,CourseType_Product_Self,CourseType_Theme_Self,CourseType_Corporate_Mandatory,CourseType_Product_Mandatory,CourseType_Theme_Mandatory
0,2018Q1,201801,201801,0.0,2018-01-01,ACCESSORIES,11.8832,577781,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
14,2018Q1,201801,201801,4.0,2018-01-01,ITEM,583.4976,577781,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
16,2018Q1,201801,201801,3.0,2018-01-01,ITEM,585.0656,577781,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
17,2018Q1,201801,201801,1.0,2018-01-01,ITEM,227.3600,577781,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
34,2018Q1,201801,201801,1.0,2018-01-01,ITEM,55.9888,577781,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3819509,2018Q4,201811,201844,0.0,2018-10-29,ACCESSORIES,14.5488,577781,C,2015-11-16,5.0,0.0,0.0,0.0,0.0,0.0
3819523,2018Q4,201811,201844,1.0,2018-10-29,ITEM,160.1600,577781,C,2015-11-16,5.0,0.0,0.0,0.0,0.0,0.0
3819530,2018Q4,201811,201844,1.0,2018-10-29,ITEM,16.8000,577781,C,2015-11-16,5.0,0.0,0.0,0.0,0.0,0.0
3824281,2018Q4,201811,201844,0.0,2018-10-29,ACCESSORIES,6.7088,577781,C,2015-11-16,5.0,0.0,0.0,0.0,0.0,0.0


In [46]:
mergenot = mergenot.sort_values(by=['USER_ID','Time Completed'])
mergenot = mergenot.bfill()
mergenot #filled with the first following available data (per user id), since at a certain point they get train hence extra info

,Fiscal Quarter (SS),Fiscal Month (SS),Fiscal Week (SS),Retail Units,Time Completed,Commodity,Sales,USER_ID,EmployeeType,HiringDate_HR,CourseType_Corporate_Self,CourseType_Product_Self,CourseType_Theme_Self,CourseType_Corporate_Mandatory,CourseType_Product_Mandatory,CourseType_Theme_Mandatory
15372,2018Q1,201801,201801,1.0,2018-01-02,ITEM,55.9888,1035,P_FT,1992-03-23,0.0,0.0,0.0,0.0,0.0,0.0
15548,2018Q1,201801,201801,1.0,2018-01-02,ITEM,111.9888,1035,P_FT,1992-03-23,0.0,0.0,0.0,0.0,0.0,0.0
15558,2018Q1,201801,201801,2.0,2018-01-02,ITEM,393.1200,1035,P_FT,1992-03-23,0.0,0.0,0.0,0.0,0.0,0.0
15614,2018Q1,201801,201801,0.0,2018-01-02,ACCESSORIES,15.6688,1035,P_FT,1992-03-23,0.0,0.0,0.0,0.0,0.0,0.0
15615,2018Q1,201801,201801,1.0,2018-01-02,ITEM,223.9888,1035,P_FT,1992-03-23,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8097156,2019Q4,201910,201942,1.0,2019-10-14,ITEM,364.0000,978685,C,2019-10-06,5.0,40.0,30.0,0.0,0.0,0.0
8109542,2019Q4,201910,201942,2.0,2019-10-15,ITEM,421.1200,978685,C,2019-10-06,5.0,40.0,30.0,0.0,0.0,0.0
8088543,2019Q4,201910,201942,0.0,2019-10-13,ACCESSORIES,13.4288,978686,C,2019-10-07,10.0,30.0,0.0,0.0,0.0,0.0
8088556,2019Q4,201910,201942,1.0,2019-10-13,ITEM,213.9200,978686,C,2019-10-07,10.0,30.0,0.0,0.0,0.0,0.0


In [47]:
mergenot[mergenot['USER_ID']== 1035].sort_values(by='Time Completed') #every user is filled correctly

,Fiscal Quarter (SS),Fiscal Month (SS),Fiscal Week (SS),Retail Units,Time Completed,Commodity,Sales,USER_ID,EmployeeType,HiringDate_HR,CourseType_Corporate_Self,CourseType_Product_Self,CourseType_Theme_Self,CourseType_Corporate_Mandatory,CourseType_Product_Mandatory,CourseType_Theme_Mandatory
15372,2018Q1,201801,201801,1.0,2018-01-02,ITEM,55.9888,1035,P_FT,1992-03-23,0.0,0.0,0.0,0.0,0.0,0.0
15548,2018Q1,201801,201801,1.0,2018-01-02,ITEM,111.9888,1035,P_FT,1992-03-23,0.0,0.0,0.0,0.0,0.0,0.0
15558,2018Q1,201801,201801,2.0,2018-01-02,ITEM,393.1200,1035,P_FT,1992-03-23,0.0,0.0,0.0,0.0,0.0,0.0
15614,2018Q1,201801,201801,0.0,2018-01-02,ACCESSORIES,15.6688,1035,P_FT,1992-03-23,0.0,0.0,0.0,0.0,0.0,0.0
15615,2018Q1,201801,201801,1.0,2018-01-02,ITEM,223.9888,1035,P_FT,1992-03-23,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8106733,2019Q4,201910,201942,0.0,2019-10-14,ACCESSORIES,20.1040,1035,P_FT,1992-03-23,20.0,70.0,140.0,0.0,95.0,40.0
8107695,2019Q4,201910,201942,1.0,2019-10-15,ITEM,111.9888,1035,P_FT,1992-03-23,20.0,70.0,140.0,0.0,95.0,40.0
8107593,2019Q4,201910,201942,0.0,2019-10-15,ACCESSORIES,22.3776,1035,P_FT,1992-03-23,20.0,70.0,140.0,0.0,95.0,40.0
8107630,2019Q4,201910,201942,1.0,2019-10-15,ITEM,167.9888,1035,P_FT,1992-03-23,20.0,70.0,140.0,0.0,95.0,40.0


In [48]:
notrain = merge[merge['USER_ID'].isin(missing2)].reset_index(drop=True)
notrain #those who never get any train, the hours in training are filled with 0 ofc, but hiring date and employee won't ever be filled

,Fiscal Quarter (SS),Fiscal Month (SS),Fiscal Week (SS),Retail Units,Time Completed,Commodity,Sales,USER_ID,EmployeeType,HiringDate_HR,CourseType_Corporate_Self,CourseType_Product_Self,CourseType_Theme_Self,CourseType_Corporate_Mandatory,CourseType_Product_Mandatory,CourseType_Theme_Mandatory
0,2018Q1,201801,201801,2.0,2018-01-01,ITEM,269.6400,588104,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
1,2018Q1,201801,201801,1.0,2018-01-01,ITEM,182.5600,592146,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
2,2018Q1,201801,201801,1.0,2018-01-01,ITEM,111.9888,592146,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
3,2018Q1,201801,201801,1.0,2018-01-01,ITEM,111.9888,592146,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
4,2018Q1,201801,201801,1.0,2018-01-01,ITEM,55.9888,592146,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799256,2019Q4,201910,201942,1.0,2019-10-15,ITEM,204.9600,737066,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
799257,2019Q4,201910,201942,0.0,2019-10-15,ACCESSORIES,24.6176,737061,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
799258,2019Q4,201910,201942,0.0,2019-10-15,ACCESSORIES,11.1888,737065,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
799259,2019Q4,201910,201942,1.0,2019-10-15,ITEM,171.3600,586217,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0


In [49]:
merge2 = mergenot.append(notrain, ignore_index= True)
merge2 = merge2.sort_values(by=['USER_ID','Time Completed'])
merge2.isnull().sum() #the only missing values are from those never get any train

Fiscal Quarter (SS)                    0
Fiscal Month (SS)                      0
Fiscal Week (SS)                       0
Retail Units                           0
Time Completed                         0
Commodity                              0
Sales                                  0
USER_ID                                0
EmployeeType                      799261
HiringDate_HR                     799261
CourseType_Corporate_Self              0
CourseType_Product_Self                0
CourseType_Theme_Self                  0
CourseType_Corporate_Mandatory         0
CourseType_Product_Mandatory           0
CourseType_Theme_Mandatory             0
dtype: int64

In [50]:
merge2.dtypes

Fiscal Quarter (SS)                       object
Fiscal Month (SS)                          int64
Fiscal Week (SS)                           int64
Retail Units                             float64
Time Completed                    datetime64[ns]
Commodity                                 object
Sales                                    float64
USER_ID                                    int64
EmployeeType                              object
HiringDate_HR                             object
CourseType_Corporate_Self                float64
CourseType_Product_Self                  float64
CourseType_Theme_Self                    float64
CourseType_Corporate_Mandatory           float64
CourseType_Product_Mandatory             float64
CourseType_Theme_Mandatory               float64
dtype: object

In [51]:
merge2['Trained'] = np.where(merge2['EmployeeType'].isnull(),0,1)
merge2= merge2.reset_index(drop=True)
merge2

,Fiscal Quarter (SS),Fiscal Month (SS),Fiscal Week (SS),Retail Units,Time Completed,Commodity,Sales,USER_ID,EmployeeType,HiringDate_HR,CourseType_Corporate_Self,CourseType_Product_Self,CourseType_Theme_Self,CourseType_Corporate_Mandatory,CourseType_Product_Mandatory,CourseType_Theme_Mandatory,Trained
0,2018Q1,201802,201807,0.0,2018-02-16,ACCESSORIES,6.2608,9,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0
1,2018Q1,201803,201809,0.0,2018-02-28,ACCESSORIES,10.0688,9,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0
2,2018Q1,201803,201809,0.0,2018-03-02,ACCESSORIES,11.1888,9,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0
3,2018Q1,201803,201810,0.0,2018-03-05,ACCESSORIES,10.0688,9,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0
4,2018Q1,201803,201811,0.0,2018-03-12,ACCESSORIES,6.7088,9,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8913289,2019Q4,201910,201942,1.0,2019-10-14,ITEM,229.8240,1000008,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0
8913290,2019Q4,201910,201942,1.0,2019-10-14,ITEM,185.9200,1000008,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0
8913291,2019Q4,201910,201942,2.0,2019-10-14,ITEM,349.4400,1000008,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0
8913292,2019Q4,201910,201942,1.0,2019-10-14,ITEM,77.6160,1000008,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0


In [52]:
merge2['EmployeeType'].value_counts(normalize=True)

P_FT    0.508981
C       0.374536
P_PT    0.116483
Name: EmployeeType, dtype: float64

In [53]:
group = merge2.drop_duplicates('USER_ID')
group['EmployeeType'].value_counts(normalize=True)

C       0.674817
P_FT    0.198539
P_PT    0.126644
Name: EmployeeType, dtype: float64

In [54]:
import random

nans = group['EmployeeType'].isna()
length = sum(nans)
replacement = random.choices(['P_FT', 'C','P_PT'], weights=[.20, .67, .13], k=length)
group.loc[nans,'EmployeeType'] = replacement

group


,Fiscal Quarter (SS),Fiscal Month (SS),Fiscal Week (SS),Retail Units,Time Completed,Commodity,Sales,USER_ID,EmployeeType,HiringDate_HR,CourseType_Corporate_Self,CourseType_Product_Self,CourseType_Theme_Self,CourseType_Corporate_Mandatory,CourseType_Product_Mandatory,CourseType_Theme_Mandatory,Trained
0,2018Q1,201802,201807,0.0,2018-02-16,ACCESSORIES,6.2608,9,C,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0
6,2018Q2,201805,201820,2.0,2018-05-19,ITEM,490.5600,10,C,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0
25,2018Q2,201805,201820,1.0,2018-05-18,ITEM,244.1600,11,P_PT,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0
31,2018Q1,201801,201801,1.0,2018-01-02,ITEM,55.9888,1035,P_FT,1992-03-23,0.0,0.0,0.0,0.0,0.0,0.0,1
2147,2018Q1,201801,201804,1.0,2018-01-25,ITEM,328.1600,1340,P_FT,2000-09-03,0.0,0.0,0.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8585658,2019Q4,201910,201942,3.0,2019-10-14,ITEM,665.2800,978671,C,2019-10-06,0.0,0.0,0.0,0.0,0.0,0.0,1
8585661,2019Q4,201910,201941,1.0,2019-10-11,ITEM,302.4000,978685,C,2019-10-06,0.0,0.0,0.0,0.0,0.0,0.0,1
8585665,2019Q4,201910,201942,0.0,2019-10-13,ACCESSORIES,13.4288,978686,C,2019-10-07,10.0,30.0,0.0,0.0,0.0,0.0,1
8585668,2018Q1,201801,201801,3.0,2018-01-01,ITEM,525.2800,1000006,C,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0


In [55]:
s = merge2['USER_ID'].map(group.set_index('USER_ID')['EmployeeType'])

In [56]:
merge2['EmployeeType']= merge2['EmployeeType'].mask(merge2['EmployeeType'].isnull(), s)
merge2

,Fiscal Quarter (SS),Fiscal Month (SS),Fiscal Week (SS),Retail Units,Time Completed,Commodity,Sales,USER_ID,EmployeeType,HiringDate_HR,CourseType_Corporate_Self,CourseType_Product_Self,CourseType_Theme_Self,CourseType_Corporate_Mandatory,CourseType_Product_Mandatory,CourseType_Theme_Mandatory,Trained
0,2018Q1,201802,201807,0.0,2018-02-16,ACCESSORIES,6.2608,9,C,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0
1,2018Q1,201803,201809,0.0,2018-02-28,ACCESSORIES,10.0688,9,C,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0
2,2018Q1,201803,201809,0.0,2018-03-02,ACCESSORIES,11.1888,9,C,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0
3,2018Q1,201803,201810,0.0,2018-03-05,ACCESSORIES,10.0688,9,C,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0
4,2018Q1,201803,201811,0.0,2018-03-12,ACCESSORIES,6.7088,9,C,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8913289,2019Q4,201910,201942,1.0,2019-10-14,ITEM,229.8240,1000008,C,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0
8913290,2019Q4,201910,201942,1.0,2019-10-14,ITEM,185.9200,1000008,C,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0
8913291,2019Q4,201910,201942,2.0,2019-10-14,ITEM,349.4400,1000008,C,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0
8913292,2019Q4,201910,201942,1.0,2019-10-14,ITEM,77.6160,1000008,C,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0


In [57]:
merge2[merge2['USER_ID']== 45119].sort_values(by='Time Completed') #check values of id from "missing", they keep coherence

,Fiscal Quarter (SS),Fiscal Month (SS),Fiscal Week (SS),Retail Units,Time Completed,Commodity,Sales,USER_ID,EmployeeType,HiringDate_HR,CourseType_Corporate_Self,CourseType_Product_Self,CourseType_Theme_Self,CourseType_Corporate_Mandatory,CourseType_Product_Mandatory,CourseType_Theme_Mandatory,Trained
24288,2019Q3,201909,201936,1.0,2019-09-06,ITEM,134.3440,45119,C,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0
24289,2019Q3,201909,201936,1.0,2019-09-06,ITEM,425.6000,45119,C,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0
24290,2019Q3,201909,201936,0.0,2019-09-06,ACCESSORIES,11.1888,45119,C,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0


In [58]:
merge2.isnull().sum()

Fiscal Quarter (SS)                    0
Fiscal Month (SS)                      0
Fiscal Week (SS)                       0
Retail Units                           0
Time Completed                         0
Commodity                              0
Sales                                  0
USER_ID                                0
EmployeeType                           0
HiringDate_HR                     799261
CourseType_Corporate_Self              0
CourseType_Product_Self                0
CourseType_Theme_Self                  0
CourseType_Corporate_Mandatory         0
CourseType_Product_Mandatory           0
CourseType_Theme_Mandatory             0
Trained                                0
dtype: int64

In [59]:
hiring = merge2.drop_duplicates('USER_ID')
hiring['HiringDate_HR'] = hiring['HiringDate_HR'].astype('datetime64[ns]')
hiring['HiringDate_HR'] = hiring['HiringDate_HR'].fillna(hiring['Time Completed'].dt.floor('d') - pd.offsets.MonthBegin(1))
hiring

,Fiscal Quarter (SS),Fiscal Month (SS),Fiscal Week (SS),Retail Units,Time Completed,Commodity,Sales,USER_ID,EmployeeType,HiringDate_HR,CourseType_Corporate_Self,CourseType_Product_Self,CourseType_Theme_Self,CourseType_Corporate_Mandatory,CourseType_Product_Mandatory,CourseType_Theme_Mandatory,Trained
0,2018Q1,201802,201807,0.0,2018-02-16,ACCESSORIES,6.2608,9,C,2018-02-01,0.0,0.0,0.0,0.0,0.0,0.0,0
6,2018Q2,201805,201820,2.0,2018-05-19,ITEM,490.5600,10,C,2018-05-01,0.0,0.0,0.0,0.0,0.0,0.0,0
25,2018Q2,201805,201820,1.0,2018-05-18,ITEM,244.1600,11,P_PT,2018-05-01,0.0,0.0,0.0,0.0,0.0,0.0,0
31,2018Q1,201801,201801,1.0,2018-01-02,ITEM,55.9888,1035,P_FT,1992-03-23,0.0,0.0,0.0,0.0,0.0,0.0,1
2147,2018Q1,201801,201804,1.0,2018-01-25,ITEM,328.1600,1340,P_FT,2000-09-03,0.0,0.0,0.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8585658,2019Q4,201910,201942,3.0,2019-10-14,ITEM,665.2800,978671,C,2019-10-06,0.0,0.0,0.0,0.0,0.0,0.0,1
8585661,2019Q4,201910,201941,1.0,2019-10-11,ITEM,302.4000,978685,C,2019-10-06,0.0,0.0,0.0,0.0,0.0,0.0,1
8585665,2019Q4,201910,201942,0.0,2019-10-13,ACCESSORIES,13.4288,978686,C,2019-10-07,10.0,30.0,0.0,0.0,0.0,0.0,1
8585668,2018Q1,201801,201801,3.0,2018-01-01,ITEM,525.2800,1000006,C,2017-12-01,0.0,0.0,0.0,0.0,0.0,0.0,0


In [60]:
s2 = merge2['USER_ID'].map(hiring.set_index('USER_ID')['HiringDate_HR'])
merge2['HiringDate_HR'] = merge2['HiringDate_HR'].astype('datetime64[ns]')

# FINAL DATASET

In [61]:
merge2['HiringDate_HR']= merge2['HiringDate_HR'].mask(merge2['HiringDate_HR'].isnull(), s2)
merge2

,Fiscal Quarter (SS),Fiscal Month (SS),Fiscal Week (SS),Retail Units,Time Completed,Commodity,Sales,USER_ID,EmployeeType,HiringDate_HR,CourseType_Corporate_Self,CourseType_Product_Self,CourseType_Theme_Self,CourseType_Corporate_Mandatory,CourseType_Product_Mandatory,CourseType_Theme_Mandatory,Trained
0,2018Q1,201802,201807,0.0,2018-02-16,ACCESSORIES,6.2608,9,C,2018-02-01,0.0,0.0,0.0,0.0,0.0,0.0,0
1,2018Q1,201803,201809,0.0,2018-02-28,ACCESSORIES,10.0688,9,C,2018-02-01,0.0,0.0,0.0,0.0,0.0,0.0,0
2,2018Q1,201803,201809,0.0,2018-03-02,ACCESSORIES,11.1888,9,C,2018-02-01,0.0,0.0,0.0,0.0,0.0,0.0,0
3,2018Q1,201803,201810,0.0,2018-03-05,ACCESSORIES,10.0688,9,C,2018-02-01,0.0,0.0,0.0,0.0,0.0,0.0,0
4,2018Q1,201803,201811,0.0,2018-03-12,ACCESSORIES,6.7088,9,C,2018-02-01,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8913289,2019Q4,201910,201942,1.0,2019-10-14,ITEM,229.8240,1000008,C,2017-12-01,0.0,0.0,0.0,0.0,0.0,0.0,0
8913290,2019Q4,201910,201942,1.0,2019-10-14,ITEM,185.9200,1000008,C,2017-12-01,0.0,0.0,0.0,0.0,0.0,0.0,0
8913291,2019Q4,201910,201942,2.0,2019-10-14,ITEM,349.4400,1000008,C,2017-12-01,0.0,0.0,0.0,0.0,0.0,0.0,0
8913292,2019Q4,201910,201942,1.0,2019-10-14,ITEM,77.6160,1000008,C,2017-12-01,0.0,0.0,0.0,0.0,0.0,0.0,0


In [62]:
merge2.isnull().sum()

Fiscal Quarter (SS)               0
Fiscal Month (SS)                 0
Fiscal Week (SS)                  0
Retail Units                      0
Time Completed                    0
Commodity                         0
Sales                             0
USER_ID                           0
EmployeeType                      0
HiringDate_HR                     0
CourseType_Corporate_Self         0
CourseType_Product_Self           0
CourseType_Theme_Self             0
CourseType_Corporate_Mandatory    0
CourseType_Product_Mandatory      0
CourseType_Theme_Mandatory        0
Trained                           0
dtype: int64

In [63]:
merge2.dtypes

Fiscal Quarter (SS)                       object
Fiscal Month (SS)                          int64
Fiscal Week (SS)                           int64
Retail Units                             float64
Time Completed                    datetime64[ns]
Commodity                                 object
Sales                                    float64
USER_ID                                    int64
EmployeeType                              object
HiringDate_HR                     datetime64[ns]
CourseType_Corporate_Self                float64
CourseType_Product_Self                  float64
CourseType_Theme_Self                    float64
CourseType_Corporate_Mandatory           float64
CourseType_Product_Mandatory             float64
CourseType_Theme_Mandatory               float64
Trained                                    int32
dtype: object

In [64]:
merge2['Year'] = merge2['Fiscal Month (SS)'].astype(str).str[:4].astype(int)
merge2['Month'] = merge2['Fiscal Month (SS)'].astype(str).str[-2:].astype(int)
merge2['Week'] = merge2['Fiscal Week (SS)'].astype(str).str[-2:].astype(int)

In [65]:
merge2

,Fiscal Quarter (SS),Fiscal Month (SS),Fiscal Week (SS),Retail Units,Time Completed,Commodity,Sales,USER_ID,EmployeeType,HiringDate_HR,CourseType_Corporate_Self,CourseType_Product_Self,CourseType_Theme_Self,CourseType_Corporate_Mandatory,CourseType_Product_Mandatory,CourseType_Theme_Mandatory,Trained,Year,Month,Week
0,2018Q1,201802,201807,0.0,2018-02-16,ACCESSORIES,6.2608,9,C,2018-02-01,0.0,0.0,0.0,0.0,0.0,0.0,0,2018,2,7
1,2018Q1,201803,201809,0.0,2018-02-28,ACCESSORIES,10.0688,9,C,2018-02-01,0.0,0.0,0.0,0.0,0.0,0.0,0,2018,3,9
2,2018Q1,201803,201809,0.0,2018-03-02,ACCESSORIES,11.1888,9,C,2018-02-01,0.0,0.0,0.0,0.0,0.0,0.0,0,2018,3,9
3,2018Q1,201803,201810,0.0,2018-03-05,ACCESSORIES,10.0688,9,C,2018-02-01,0.0,0.0,0.0,0.0,0.0,0.0,0,2018,3,10
4,2018Q1,201803,201811,0.0,2018-03-12,ACCESSORIES,6.7088,9,C,2018-02-01,0.0,0.0,0.0,0.0,0.0,0.0,0,2018,3,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8913289,2019Q4,201910,201942,1.0,2019-10-14,ITEM,229.8240,1000008,C,2017-12-01,0.0,0.0,0.0,0.0,0.0,0.0,0,2019,10,42
8913290,2019Q4,201910,201942,1.0,2019-10-14,ITEM,185.9200,1000008,C,2017-12-01,0.0,0.0,0.0,0.0,0.0,0.0,0,2019,10,42
8913291,2019Q4,201910,201942,2.0,2019-10-14,ITEM,349.4400,1000008,C,2017-12-01,0.0,0.0,0.0,0.0,0.0,0.0,0,2019,10,42
8913292,2019Q4,201910,201942,1.0,2019-10-14,ITEM,77.6160,1000008,C,2017-12-01,0.0,0.0,0.0,0.0,0.0,0.0,0,2019,10,42


In [66]:
from datetime import datetime

merge2['HiringDate_HR'] = merge2['HiringDate_HR'].astype('object')
merge2['HiringDate_HR'] = (datetime.now()  - pd.to_datetime(merge2['HiringDate_HR'],infer_datetime_format=True)).dt.days
merge2

,Fiscal Quarter (SS),Fiscal Month (SS),Fiscal Week (SS),Retail Units,Time Completed,Commodity,Sales,USER_ID,EmployeeType,HiringDate_HR,CourseType_Corporate_Self,CourseType_Product_Self,CourseType_Theme_Self,CourseType_Corporate_Mandatory,CourseType_Product_Mandatory,CourseType_Theme_Mandatory,Trained,Year,Month,Week
0,2018Q1,201802,201807,0.0,2018-02-16,ACCESSORIES,6.2608,9,C,824,0.0,0.0,0.0,0.0,0.0,0.0,0,2018,2,7
1,2018Q1,201803,201809,0.0,2018-02-28,ACCESSORIES,10.0688,9,C,824,0.0,0.0,0.0,0.0,0.0,0.0,0,2018,3,9
2,2018Q1,201803,201809,0.0,2018-03-02,ACCESSORIES,11.1888,9,C,824,0.0,0.0,0.0,0.0,0.0,0.0,0,2018,3,9
3,2018Q1,201803,201810,0.0,2018-03-05,ACCESSORIES,10.0688,9,C,824,0.0,0.0,0.0,0.0,0.0,0.0,0,2018,3,10
4,2018Q1,201803,201811,0.0,2018-03-12,ACCESSORIES,6.7088,9,C,824,0.0,0.0,0.0,0.0,0.0,0.0,0,2018,3,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8913289,2019Q4,201910,201942,1.0,2019-10-14,ITEM,229.8240,1000008,C,886,0.0,0.0,0.0,0.0,0.0,0.0,0,2019,10,42
8913290,2019Q4,201910,201942,1.0,2019-10-14,ITEM,185.9200,1000008,C,886,0.0,0.0,0.0,0.0,0.0,0.0,0,2019,10,42
8913291,2019Q4,201910,201942,2.0,2019-10-14,ITEM,349.4400,1000008,C,886,0.0,0.0,0.0,0.0,0.0,0.0,0,2019,10,42
8913292,2019Q4,201910,201942,1.0,2019-10-14,ITEM,77.6160,1000008,C,886,0.0,0.0,0.0,0.0,0.0,0.0,0,2019,10,42


# regressions trials

In [67]:
merge3 = merge2

In [68]:
mergeITEM = merge3[merge3['Commodity'] == 'ITEM']
mergeITEM

,Fiscal Quarter (SS),Fiscal Month (SS),Fiscal Week (SS),Retail Units,Time Completed,Commodity,Sales,USER_ID,EmployeeType,HiringDate_HR,CourseType_Corporate_Self,CourseType_Product_Self,CourseType_Theme_Self,CourseType_Corporate_Mandatory,CourseType_Product_Mandatory,CourseType_Theme_Mandatory,Trained,Year,Month,Week
5,2018Q2,201804,201816,1.0,2018-04-20,ITEM,160.1600,9,C,824,0.0,0.0,0.0,0.0,0.0,0.0,0,2018,4,16
6,2018Q2,201805,201820,2.0,2018-05-19,ITEM,490.5600,10,C,735,0.0,0.0,0.0,0.0,0.0,0.0,0,2018,5,20
8,2018Q2,201805,201821,1.0,2018-05-20,ITEM,182.5600,10,C,735,0.0,0.0,0.0,0.0,0.0,0.0,0,2018,5,21
9,2018Q2,201805,201821,1.0,2018-05-20,ITEM,100.7888,10,C,735,0.0,0.0,0.0,0.0,0.0,0.0,0,2018,5,21
10,2018Q2,201805,201821,1.0,2018-05-21,ITEM,189.2800,10,C,735,0.0,0.0,0.0,0.0,0.0,0.0,0,2018,5,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8913289,2019Q4,201910,201942,1.0,2019-10-14,ITEM,229.8240,1000008,C,886,0.0,0.0,0.0,0.0,0.0,0.0,0,2019,10,42
8913290,2019Q4,201910,201942,1.0,2019-10-14,ITEM,185.9200,1000008,C,886,0.0,0.0,0.0,0.0,0.0,0.0,0,2019,10,42
8913291,2019Q4,201910,201942,2.0,2019-10-14,ITEM,349.4400,1000008,C,886,0.0,0.0,0.0,0.0,0.0,0.0,0,2019,10,42
8913292,2019Q4,201910,201942,1.0,2019-10-14,ITEM,77.6160,1000008,C,886,0.0,0.0,0.0,0.0,0.0,0.0,0,2019,10,42


In [69]:
mergeACCESSORY = merge3[merge3['Commodity'] == 'ACCESSORIES']
mergeACCESSORY

,Fiscal Quarter (SS),Fiscal Month (SS),Fiscal Week (SS),Retail Units,Time Completed,Commodity,Sales,USER_ID,EmployeeType,HiringDate_HR,CourseType_Corporate_Self,CourseType_Product_Self,CourseType_Theme_Self,CourseType_Corporate_Mandatory,CourseType_Product_Mandatory,CourseType_Theme_Mandatory,Trained,Year,Month,Week
0,2018Q1,201802,201807,0.0,2018-02-16,ACCESSORIES,6.2608,9,C,824,0.0,0.0,0.0,0.0,0.0,0.0,0,2018,2,7
1,2018Q1,201803,201809,0.0,2018-02-28,ACCESSORIES,10.0688,9,C,824,0.0,0.0,0.0,0.0,0.0,0.0,0,2018,3,9
2,2018Q1,201803,201809,0.0,2018-03-02,ACCESSORIES,11.1888,9,C,824,0.0,0.0,0.0,0.0,0.0,0.0,0,2018,3,9
3,2018Q1,201803,201810,0.0,2018-03-05,ACCESSORIES,10.0688,9,C,824,0.0,0.0,0.0,0.0,0.0,0.0,0,2018,3,10
4,2018Q1,201803,201811,0.0,2018-03-12,ACCESSORIES,6.7088,9,C,824,0.0,0.0,0.0,0.0,0.0,0.0,0,2018,3,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8913264,2019Q4,201910,201942,0.0,2019-10-14,ACCESSORIES,20.1376,1000008,C,886,0.0,0.0,0.0,0.0,0.0,0.0,0,2019,10,42
8913265,2019Q4,201910,201942,0.0,2019-10-14,ACCESSORIES,50.3440,1000008,C,886,0.0,0.0,0.0,0.0,0.0,0.0,0,2019,10,42
8913266,2019Q4,201910,201942,0.0,2019-10-14,ACCESSORIES,10.0688,1000008,C,886,0.0,0.0,0.0,0.0,0.0,0.0,0,2019,10,42
8913267,2019Q4,201910,201942,0.0,2019-10-14,ACCESSORIES,22.3776,1000008,C,886,0.0,0.0,0.0,0.0,0.0,0.0,0,2019,10,42


In [114]:
f = {'CourseType_Corporate_Self':['max'], 'CourseType_Product_Self':['max'], 'CourseType_Theme_Self':['max'],'CourseType_Corporate_Mandatory':['max'],\
    'CourseType_Product_Mandatory':['max'],'CourseType_Theme_Mandatory':['max'],'Sales':['sum'],'Trained':['max'],'HiringDate_HR':['max']}

linear1= mergeACCESSORY.groupby(['USER_ID','Year','Week','EmployeeType']).agg(f)

linear1

CourseType_Corporate_Self  \
                                                     max   
USER_ID Year Week EmployeeType                             
9       2018 7    C                                  0.0   
             9    C                                  0.0   
             10   C                                  0.0   
             11   C                                  0.0   
10      2018 20   C                                  0.0   
...                                                  ...   
1000008 2019 38   C                                  0.0   
             39   C                                  0.0   
             40   C                                  0.0   
             41   C                                  0.0   
             42   C                                  0.0   

                               CourseType_Product_Self CourseType_Theme_Self  \
                                                   max                   max   
USER_ID Year Week EmployeeType                                                 
9       2018 7    C                                0.0                   0.0   
             9    C                                0.0                   0.0   
             10   C                                0.0                   0.0   
             11   C                                0.0                   0.0   
10      2018 20   C                                0.0                   0.0   
...                                                ...                   ...   
1000008 2019 38   C                                0.0                   0.0   
             39   C                                0.0                   0.0   
             40   C                                0.0                   0.0   
             41   C                                0.0                   0.0   
             42   C                                0.0                   0.0   

                               CourseType_Corporate_Mandatory  \
                                                          max   
USER_ID Year Week EmployeeType                                  
9       2018 7    C                                       0.0   
             9    C                                       0.0   
             10   C                                       0.0   
             11   C                                       0.0   
10      2018 20   C                                       0.0   
...                                                       ...   
1000008 2019 38   C                                       0.0   
             39   C                                       0.0   
             40   C                                       0.0   
             41   C                                       0.0   
             42   C                                       0.0   

                               CourseType_Product_Mandatory  \
                                                        max   
USER_ID Year Week EmployeeType                                
9       2018 7    C                                     0.0   
             9    C                                     0.0   
             10   C                                     0.0   
             11   C                                     0.0   
10      2018 20   C                                     0.0   
...                                                     ...   
1000008 2019 38   C                                     0.0   
             39   C                                     0.0   
             40   C                                     0.0   
             41   C                                     0.0   
             42   C                                     0.0   

                               CourseType_Theme_Mandatory       Sales Trained  \
                                                      max         sum     max   
USER_ID Year Week EmployeeType                                                  
9       2018 7    C             

# LINEAR REGRESSION BY WEEK FOR ACCESSORIES

In [115]:
linear1 = linear1.reset_index()
linear1

,USER_ID,Year,Week,EmployeeType,CourseType_Corporate_Self,CourseType_Product_Self,CourseType_Theme_Self,CourseType_Corporate_Mandatory,CourseType_Product_Mandatory,CourseType_Theme_Mandatory,Sales,Trained,HiringDate_HR
,,,,,max,max,max,max,max,max,sum,max,max
0,9,2018,7,C,0.0,0.0,0.0,0.0,0.0,0.0,6.2608,0,824
1,9,2018,9,C,0.0,0.0,0.0,0.0,0.0,0.0,21.2576,0,824
2,9,2018,10,C,0.0,0.0,0.0,0.0,0.0,0.0,10.0688,0,824
3,9,2018,11,C,0.0,0.0,0.0,0.0,0.0,0.0,6.7088,0,824
4,10,2018,20,C,0.0,0.0,0.0,0.0,0.0,0.0,8.9488,0,735
...,...,...,...,...,...,...,...,...,...,...,...,...,...
492868,1000008,2019,38,C,0.0,0.0,0.0,0.0,0.0,0.0,37992.9648,0,886
492869,1000008,2019,39,C,0.0,0.0,0.0,0.0,0.0,0.0,36192.8112,0,886
492870,1000008,2019,40,C,0.0,0.0,0.0,0.0,0.0,0.0,33515.8544,0,886


In [72]:
k = {'CourseType_Corporate_Self':['max'], 'CourseType_Product_Self':['max'], 'CourseType_Theme_Self':['max'],'CourseType_Corporate_Mandatory':['max'],\
    'CourseType_Product_Mandatory':['max'],'CourseType_Theme_Mandatory':['max'],'Sales':['sum'],'Retail Units':['sum'],'Trained':['max'],'HiringDate_HR':['max']}

linear2= mergeITEM.groupby(['USER_ID','Year','Week','EmployeeType']).agg(k)

linear2

CourseType_Corporate_Self  \
                                                     max   
USER_ID Year Week EmployeeType                             
9       2018 16   C                                  0.0   
10      2018 20   C                                  0.0   
             21   C                                  0.0   
             22   C                                  0.0   
11      2018 20   P_PT                               0.0   
...                                                  ...   
1000008 2019 38   C                                  0.0   
             39   C                                  0.0   
             40   C                                  0.0   
             41   C                                  0.0   
             42   C                                  0.0   

                               CourseType_Product_Self CourseType_Theme_Self  \
                                                   max                   max   
USER_ID Year Week EmployeeType                                                 
9       2018 16   C                                0.0                   0.0   
10      2018 20   C                                0.0                   0.0   
             21   C                                0.0                   0.0   
             22   C                                0.0                   0.0   
11      2018 20   P_PT                             0.0                   0.0   
...                                                ...                   ...   
1000008 2019 38   C                                0.0                   0.0   
             39   C                                0.0                   0.0   
             40   C                                0.0                   0.0   
             41   C                                0.0                   0.0   
             42   C                                0.0                   0.0   

                               CourseType_Corporate_Mandatory  \
                                                          max   
USER_ID Year Week EmployeeType                                  
9       2018 16   C                                       0.0   
10      2018 20   C                                       0.0   
             21   C                                       0.0   
             22   C                                       0.0   
11      2018 20   P_PT                                    0.0   
...                                                       ...   
1000008 2019 38   C                                       0.0   
             39   C                                       0.0   
             40   C                                       0.0   
             41   C                                       0.0   
             42   C                                       0.0   

                               CourseType_Product_Mandatory  \
                                                        max   
USER_ID Year Week EmployeeType                                
9       2018 16   C                                     0.0   
10      2018 20   C                                     0.0   
             21   C                                     0.0   
             22   C                                     0.0   
11      2018 20   P_PT                                  0.0   
...                                                     ...   
1000008 2019 38   C                                     0.0   
             39   C                                     0.0   
             40   C                                     0.0   
             41   C                                     0.0   
             42   C                                     0.0   

                               CourseType_Theme_Mandatory       Sales  \
                                                      max         sum   
USER_ID Year Week EmployeeType                                          
9       2018 16   C                                   0.

# LINEAR REGRESSION BY WEEKS FOR ITEM

In [73]:
linear2 = linear2.reset_index()
linear2

,USER_ID,Year,Week,EmployeeType,CourseType_Corporate_Self,CourseType_Product_Self,CourseType_Theme_Self,CourseType_Corporate_Mandatory,CourseType_Product_Mandatory,CourseType_Theme_Mandatory,Sales,Retail Units,Trained,HiringDate_HR
,,,,,max,max,max,max,max,max,sum,sum,max,max
0,9,2018,16,C,0.0,0.0,0.0,0.0,0.0,0.0,160.1600,1.0,0,824
1,10,2018,20,C,0.0,0.0,0.0,0.0,0.0,0.0,490.5600,2.0,0,735
2,10,2018,21,C,0.0,0.0,0.0,0.0,0.0,0.0,950.8576,6.0,0,735
3,10,2018,22,C,0.0,0.0,0.0,0.0,0.0,0.0,1375.3936,9.0,0,735
4,11,2018,20,P_PT,0.0,0.0,0.0,0.0,0.0,0.0,589.1200,3.0,0,735
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
723670,1000008,2019,38,C,0.0,0.0,0.0,0.0,0.0,0.0,35741.2048,266.0,0,886
723671,1000008,2019,39,C,0.0,0.0,0.0,0.0,0.0,0.0,39886.9184,264.0,0,886
723672,1000008,2019,40,C,0.0,0.0,0.0,0.0,0.0,0.0,42666.9376,269.0,0,886


In [74]:
linear2['USER_ID'].value_counts()

584595    88
300416    88
963364    88
582395    88
527743    88
          ..
978615     1
960013     1
741377     1
962749     1
506103     1
Name: USER_ID, Length: 25832, dtype: int64

In [75]:
linear2[linear2['USER_ID']== 584595].head(50)

,USER_ID,Year,Week,EmployeeType,CourseType_Corporate_Self,CourseType_Product_Self,CourseType_Theme_Self,CourseType_Corporate_Mandatory,CourseType_Product_Mandatory,CourseType_Theme_Mandatory,Sales,Retail Units,Trained,HiringDate_HR
,,,,,max,max,max,max,max,max,sum,sum,max,max
206047,584595,2018,1,P_FT,0.0,0.0,0.0,0.0,0.0,0.0,4146.8784,23.0,1,1476
206048,584595,2018,2,P_FT,0.0,0.0,0.0,0.0,0.0,0.0,4719.4112,28.0,1,1476
206049,584595,2018,3,P_FT,0.0,0.0,0.0,0.0,0.0,0.0,4477.2560,22.0,1,1476
206050,584595,2018,4,P_FT,0.0,0.0,0.0,0.0,0.0,0.0,1802.6400,10.0,1,1476
206051,584595,2018,5,P_FT,0.0,0.0,0.0,0.0,0.0,0.0,2950.6400,16.0,1,1476
206052,584595,2018,6,P_FT,0.0,0.0,0.0,0.0,0.0,0.0,5043.3600,27.0,1,1476
206053,584595,2018,7,P_FT,0.0,0.0,0.0,0.0,0.0,0.0,6600.0256,32.0,1,1476
206054,584595,2018,8,P_FT,0.0,0.0,0.0,0.0,0.0,0.0,6017.9840,30.0,1,1476
206055,584595,2018,9,P_FT,0.0,0.0,0.0,0.0,0.0,0.0,3970.1760,19.0,1,1476


In [76]:
linear2.columns = ['USER_ID','Year','Week','EmployeeType','CourseType_Corporate_Self','CourseType_Product_Self',\
                   'CourseType_Theme_Self','CourseType_Corporate_Mandatory','CourseType_Product_Mandatory','CourseType_Theme_Mandatory',\
                   'Sales','Retail_Units','Trained','HiringDate_HR']
linear2

,USER_ID,Year,Week,EmployeeType,CourseType_Corporate_Self,CourseType_Product_Self,CourseType_Theme_Self,CourseType_Corporate_Mandatory,CourseType_Product_Mandatory,CourseType_Theme_Mandatory,Sales,Retail_Units,Trained,HiringDate_HR
0,9,2018,16,C,0.0,0.0,0.0,0.0,0.0,0.0,160.1600,1.0,0,824
1,10,2018,20,C,0.0,0.0,0.0,0.0,0.0,0.0,490.5600,2.0,0,735
2,10,2018,21,C,0.0,0.0,0.0,0.0,0.0,0.0,950.8576,6.0,0,735
3,10,2018,22,C,0.0,0.0,0.0,0.0,0.0,0.0,1375.3936,9.0,0,735
4,11,2018,20,P_PT,0.0,0.0,0.0,0.0,0.0,0.0,589.1200,3.0,0,735
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
723670,1000008,2019,38,C,0.0,0.0,0.0,0.0,0.0,0.0,35741.2048,266.0,0,886
723671,1000008,2019,39,C,0.0,0.0,0.0,0.0,0.0,0.0,39886.9184,264.0,0,886
723672,1000008,2019,40,C,0.0,0.0,0.0,0.0,0.0,0.0,42666.9376,269.0,0,886
723673,1000008,2019,41,C,0.0,0.0,0.0,0.0,0.0,0.0,41969.2336,247.0,0,886


In [77]:
linear2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 723675 entries, 0 to 723674
Data columns (total 14 columns):
USER_ID                           723675 non-null int64
Year                              723675 non-null int64
Week                              723675 non-null int64
EmployeeType                      723675 non-null object
CourseType_Corporate_Self         723675 non-null float64
CourseType_Product_Self           723675 non-null float64
CourseType_Theme_Self             723675 non-null float64
CourseType_Corporate_Mandatory    723675 non-null float64
CourseType_Product_Mandatory      723675 non-null float64
CourseType_Theme_Mandatory        723675 non-null float64
Sales                             723675 non-null float64
Retail_Units                      723675 non-null float64
Trained                           723675 non-null int32
HiringDate_HR                     723675 non-null int64
dtypes: float64(8), int32(1), int64(4), object(1)
memory usage: 74.5+ MB


In [78]:
linear2=pd.get_dummies(linear2, columns=['EmployeeType'])
linear2=pd.get_dummies(linear2, columns=['Week'])
linear2=pd.get_dummies(linear2, columns=['Year'])

linear2

,USER_ID,CourseType_Corporate_Self,CourseType_Product_Self,CourseType_Theme_Self,CourseType_Corporate_Mandatory,CourseType_Product_Mandatory,CourseType_Theme_Mandatory,Sales,Retail_Units,Trained,...,Week_45,Week_46,Week_47,Week_48,Week_49,Week_50,Week_51,Week_52,Year_2018,Year_2019
0,9,0.0,0.0,0.0,0.0,0.0,0.0,160.1600,1.0,0,...,0,0,0,0,0,0,0,0,1,0
1,10,0.0,0.0,0.0,0.0,0.0,0.0,490.5600,2.0,0,...,0,0,0,0,0,0,0,0,1,0
2,10,0.0,0.0,0.0,0.0,0.0,0.0,950.8576,6.0,0,...,0,0,0,0,0,0,0,0,1,0
3,10,0.0,0.0,0.0,0.0,0.0,0.0,1375.3936,9.0,0,...,0,0,0,0,0,0,0,0,1,0
4,11,0.0,0.0,0.0,0.0,0.0,0.0,589.1200,3.0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
723670,1000008,0.0,0.0,0.0,0.0,0.0,0.0,35741.2048,266.0,0,...,0,0,0,0,0,0,0,0,0,1
723671,1000008,0.0,0.0,0.0,0.0,0.0,0.0,39886.9184,264.0,0,...,0,0,0,0,0,0,0,0,0,1
723672,1000008,0.0,0.0,0.0,0.0,0.0,0.0,42666.9376,269.0,0,...,0,0,0,0,0,0,0,0,0,1
723673,1000008,0.0,0.0,0.0,0.0,0.0,0.0,41969.2336,247.0,0,...,0,0,0,0,0,0,0,0,0,1


In [79]:
linear2['SUMTRAIN'] = linear2.loc[:, 'CourseType_Corporate_Self':'CourseType_Theme_Mandatory'].sum(1)

In [80]:
linear2['diff'] = linear2.groupby('USER_ID')['SUMTRAIN'].diff()

In [81]:
linear22 = linear2

In [82]:
def nan_with_cond(linear22):
    if ~np.isnan(linear22['diff']) : return linear22['diff']
    else: 
        if linear22[linear22['SUMTRAIN'] != 0] : return 0
        else : return 1

linear22['diff']= linear22.apply(nan_with_cond , axis=1)

In [83]:
def f(row):
    if row['diff'] == 0:
        val = 0
    else:
        val = 1
    return val

In [84]:
linear22['TRAIN'] = linear22.apply(f, axis=1)

In [85]:
linear22['diff2'] = linear22.groupby('USER_ID')['TRAIN'].diff()

def nan_with_cond2(linear22):
    if ~np.isnan(linear22['diff2']) : return linear22['diff2']
    else: 
        if linear22[linear22['TRAIN'] != 0] : return 0
        else : return 1

linear22['diff2']= linear22.apply(nan_with_cond2 , axis=1)

In [86]:
def per(row):
    if row['diff2'] == -1:
        val = 1
    else:
        val = 0
    return val

linear22['diff2'] = linear22.apply(per, axis=1)

In [87]:
linear22.loc[linear22['diff2'] == 1, 'TRAIN'] = 1

In [88]:
linear22[linear22['USER_ID']== 584595].head(50)

,USER_ID,CourseType_Corporate_Self,CourseType_Product_Self,CourseType_Theme_Self,CourseType_Corporate_Mandatory,CourseType_Product_Mandatory,CourseType_Theme_Mandatory,Sales,Retail_Units,Trained,...,Week_49,Week_50,Week_51,Week_52,Year_2018,Year_2019,SUMTRAIN,diff,TRAIN,diff2
206047,584595,0.0,0.0,0.0,0.0,0.0,0.0,4146.8784,23.0,1,...,0,0,0,0,1,0,0.0,0.0,0,0
206048,584595,0.0,0.0,0.0,0.0,0.0,0.0,4719.4112,28.0,1,...,0,0,0,0,1,0,0.0,0.0,0,0
206049,584595,0.0,0.0,0.0,0.0,0.0,0.0,4477.2560,22.0,1,...,0,0,0,0,1,0,0.0,0.0,0,0
206050,584595,0.0,0.0,0.0,0.0,0.0,0.0,1802.6400,10.0,1,...,0,0,0,0,1,0,0.0,0.0,0,0
206051,584595,0.0,0.0,0.0,0.0,0.0,0.0,2950.6400,16.0,1,...,0,0,0,0,1,0,0.0,0.0,0,0
206052,584595,0.0,0.0,0.0,0.0,0.0,0.0,5043.3600,27.0,1,...,0,0,0,0,1,0,0.0,0.0,0,0
206053,584595,0.0,0.0,0.0,0.0,0.0,0.0,6600.0256,32.0,1,...,0,0,0,0,1,0,0.0,0.0,0,0
206054,584595,0.0,0.0,0.0,0.0,0.0,0.0,6017.9840,30.0,1,...,0,0,0,0,1,0,0.0,0.0,0,0
206055,584595,0.0,0.0,0.0,0.0,0.0,0.0,3970.1760,19.0,1,...,0,0,0,0,1,0,0.0,0.0,0,0
206056,584595,0.0,0.0,0.0,0.0,0.0,0.0,3580.2816,17.0,1,...,0,0,0,0,1,0,0.0,0.0,0,0


In [89]:
linear22['Sales_ln'] = np.log(linear22['Sales'])
linear22['SUMTRAIN_ln'] = np.log(linear22['SUMTRAIN']+5)
linear22['HiringDate_HR_ln'] = np.log(linear22['HiringDate_HR'])

columns=['CourseType_Corporate_Self','CourseType_Product_Self','CourseType_Theme_Self','CourseType_Corporate_Mandatory','CourseType_Product_Mandatory','CourseType_Theme_Mandatory']

for i in columns:
    linear22[i+'_ln'] = np.log(linear22[i]+5)

In [101]:
def per2(row):
    if row['SUMTRAIN'] == 0 :
        val = 0
    else:
        val = 1
    return val

linear22['TRAIN2'] = linear22.apply(per2, axis=1)

In [102]:
linear22[linear22['USER_ID']== 584595].head(50)

,USER_ID,CourseType_Corporate_Self,CourseType_Product_Self,CourseType_Theme_Self,CourseType_Corporate_Mandatory,CourseType_Product_Mandatory,CourseType_Theme_Mandatory,Sales,Retail_Units,Trained,...,Sales_ln,SUMTRAIN_ln,HiringDate_HR_ln,CourseType_Corporate_Self_ln,CourseType_Product_Self_ln,CourseType_Theme_Self_ln,CourseType_Corporate_Mandatory_ln,CourseType_Product_Mandatory_ln,CourseType_Theme_Mandatory_ln,TRAIN2
206047,584595,0.0,0.0,0.0,0.0,0.0,0.0,4146.8784,23.0,1,...,8.330111,1.609438,7.297091,1.609438,1.609438,1.609438,1.609438,1.609438,1.609438,0
206048,584595,0.0,0.0,0.0,0.0,0.0,0.0,4719.4112,28.0,1,...,8.459439,1.609438,7.297091,1.609438,1.609438,1.609438,1.609438,1.609438,1.609438,0
206049,584595,0.0,0.0,0.0,0.0,0.0,0.0,4477.2560,22.0,1,...,8.406766,1.609438,7.297091,1.609438,1.609438,1.609438,1.609438,1.609438,1.609438,0
206050,584595,0.0,0.0,0.0,0.0,0.0,0.0,1802.6400,10.0,1,...,7.497008,1.609438,7.297091,1.609438,1.609438,1.609438,1.609438,1.609438,1.609438,0
206051,584595,0.0,0.0,0.0,0.0,0.0,0.0,2950.6400,16.0,1,...,7.989777,1.609438,7.297091,1.609438,1.609438,1.609438,1.609438,1.609438,1.609438,0
206052,584595,0.0,0.0,0.0,0.0,0.0,0.0,5043.3600,27.0,1,...,8.525828,1.609438,7.297091,1.609438,1.609438,1.609438,1.609438,1.609438,1.609438,0
206053,584595,0.0,0.0,0.0,0.0,0.0,0.0,6600.0256,32.0,1,...,8.794829,1.609438,7.297091,1.609438,1.609438,1.609438,1.609438,1.609438,1.609438,0
206054,584595,0.0,0.0,0.0,0.0,0.0,0.0,6017.9840,30.0,1,...,8.702508,1.609438,7.297091,1.609438,1.609438,1.609438,1.609438,1.609438,1.609438,0
206055,584595,0.0,0.0,0.0,0.0,0.0,0.0,3970.1760,19.0,1,...,8.286566,1.609438,7.297091,1.609438,1.609438,1.609438,1.609438,1.609438,1.609438,0
206056,584595,0.0,0.0,0.0,0.0,0.0,0.0,3580.2816,17.0,1,...,8.183197,1.609438,7.297091,1.609438,1.609438,1.609438,1.609438,1.609438,1.609438,0


In [107]:
linear22

,USER_ID,CourseType_Corporate_Self,CourseType_Product_Self,CourseType_Theme_Self,CourseType_Corporate_Mandatory,CourseType_Product_Mandatory,CourseType_Theme_Mandatory,Sales,Retail_Units,Trained,...,Sales_ln,SUMTRAIN_ln,HiringDate_HR_ln,CourseType_Corporate_Self_ln,CourseType_Product_Self_ln,CourseType_Theme_Self_ln,CourseType_Corporate_Mandatory_ln,CourseType_Product_Mandatory_ln,CourseType_Theme_Mandatory_ln,TRAIN2
0,9,0.0,0.0,0.0,0.0,0.0,0.0,160.1600,1.0,0,...,5.076173,1.609438,6.714171,1.609438,1.609438,1.609438,1.609438,1.609438,1.609438,0
1,10,0.0,0.0,0.0,0.0,0.0,0.0,490.5600,2.0,0,...,6.195548,1.609438,6.599870,1.609438,1.609438,1.609438,1.609438,1.609438,1.609438,0
2,10,0.0,0.0,0.0,0.0,0.0,0.0,950.8576,6.0,0,...,6.857364,1.609438,6.599870,1.609438,1.609438,1.609438,1.609438,1.609438,1.609438,0
3,10,0.0,0.0,0.0,0.0,0.0,0.0,1375.3936,9.0,0,...,7.226495,1.609438,6.599870,1.609438,1.609438,1.609438,1.609438,1.609438,1.609438,0
4,11,0.0,0.0,0.0,0.0,0.0,0.0,589.1200,3.0,0,...,6.378630,1.609438,6.599870,1.609438,1.609438,1.609438,1.609438,1.609438,1.609438,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
723670,1000008,0.0,0.0,0.0,0.0,0.0,0.0,35741.2048,266.0,0,...,10.484059,1.609438,6.786717,1.609438,1.609438,1.609438,1.609438,1.609438,1.609438,0
723671,1000008,0.0,0.0,0.0,0.0,0.0,0.0,39886.9184,264.0,0,...,10.593804,1.609438,6.786717,1.609438,1.609438,1.609438,1.609438,1.609438,1.609438,0
723672,1000008,0.0,0.0,0.0,0.0,0.0,0.0,42666.9376,269.0,0,...,10.661180,1.609438,6.786717,1.609438,1.609438,1.609438,1.609438,1.609438,1.609438,0
723673,1000008,0.0,0.0,0.0,0.0,0.0,0.0,41969.2336,247.0,0,...,10.644692,1.609438,6.786717,1.609438,1.609438,1.609438,1.609438,1.609438,1.609438,0


In [108]:
linear30 = linear22

In [111]:
linear30 = linear30.drop(['CourseType_Corporate_Self','CourseType_Product_Self','CourseType_Theme_Self','CourseType_Corporate_Mandatory','CourseType_Product_Mandatory','CourseType_Theme_Mandatory','Sales'], axis=1)

In [91]:
from statsmodels.compat import lzip
import statsmodels.api as sm
from statsmodels.formula.api import ols

# ITEM

In [161]:
model2 = ols('Sales_ln ~ CourseType_Corporate_Self_ln + CourseType_Product_Self_ln + CourseType_Theme_Self_ln +\
                            CourseType_Corporate_Mandatory_ln + CourseType_Product_Mandatory_ln +\
                            CourseType_Theme_Mandatory_ln + TRAIN2 + HiringDate_HR_ln + EmployeeType_C + EmployeeType_P_FT +Week_2+Week_3+\
                            Week_4+Week_5+Week_6+Week_7+Week_8+Week_9+Week_10+Week_11+Week_12+Week_13+Week_14+Week_15+Week_16+Week_17+\
                            Week_18+Week_19+Week_20+Week_21+Week_22+Week_23+Week_24+Week_25+Week_26+Week_27+Week_28+Week_29+Week_30+Week_31+\
                            Week_32+Week_33+Week_34+Week_35+Week_36+Week_37+Week_38+Week_39+Week_40+Week_41+Week_42+Week_43+Week_44+\
                            Week_45+Week_46+Week_47+Week_48+Week_49+Week_50+Week_51+Week_52+Year_2019'\
             , data=linear22).fit()

print(model2.summary())

                            OLS Regression Results                            
Dep. Variable:               Sales_ln   R-squared:                       0.255
Model:                            OLS   Adj. R-squared:                  0.255
Method:                 Least Squares   F-statistic:                     3991.
Date:                Tue, 05 May 2020   Prob (F-statistic):               0.00
Time:                        22:39:26   Log-Likelihood:            -9.7570e+05
No. Observations:              723675   AIC:                         1.952e+06
Df Residuals:                  723612   BIC:                         1.952e+06
Df Model:                          62                                         
Covariance Type:            nonrobust                                         
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Interc

# ACCESSORIES

In [116]:
linear1

,USER_ID,Year,Week,EmployeeType,CourseType_Corporate_Self,CourseType_Product_Self,CourseType_Theme_Self,CourseType_Corporate_Mandatory,CourseType_Product_Mandatory,CourseType_Theme_Mandatory,Sales,Trained,HiringDate_HR
,,,,,max,max,max,max,max,max,sum,max,max
0,9,2018,7,C,0.0,0.0,0.0,0.0,0.0,0.0,6.2608,0,824
1,9,2018,9,C,0.0,0.0,0.0,0.0,0.0,0.0,21.2576,0,824
2,9,2018,10,C,0.0,0.0,0.0,0.0,0.0,0.0,10.0688,0,824
3,9,2018,11,C,0.0,0.0,0.0,0.0,0.0,0.0,6.7088,0,824
4,10,2018,20,C,0.0,0.0,0.0,0.0,0.0,0.0,8.9488,0,735
...,...,...,...,...,...,...,...,...,...,...,...,...,...
492868,1000008,2019,38,C,0.0,0.0,0.0,0.0,0.0,0.0,37992.9648,0,886
492869,1000008,2019,39,C,0.0,0.0,0.0,0.0,0.0,0.0,36192.8112,0,886
492870,1000008,2019,40,C,0.0,0.0,0.0,0.0,0.0,0.0,33515.8544,0,886


In [117]:
linear1.columns = ['USER_ID','Year','Week','EmployeeType','CourseType_Corporate_Self','CourseType_Product_Self',\
                   'CourseType_Theme_Self','CourseType_Corporate_Mandatory','CourseType_Product_Mandatory','CourseType_Theme_Mandatory',\
                   'Sales','Trained', 'HiringDate_HR']

linear1

,USER_ID,Year,Week,EmployeeType,CourseType_Corporate_Self,CourseType_Product_Self,CourseType_Theme_Self,CourseType_Corporate_Mandatory,CourseType_Product_Mandatory,CourseType_Theme_Mandatory,Sales,Trained,HiringDate_HR
0,9,2018,7,C,0.0,0.0,0.0,0.0,0.0,0.0,6.2608,0,824
1,9,2018,9,C,0.0,0.0,0.0,0.0,0.0,0.0,21.2576,0,824
2,9,2018,10,C,0.0,0.0,0.0,0.0,0.0,0.0,10.0688,0,824
3,9,2018,11,C,0.0,0.0,0.0,0.0,0.0,0.0,6.7088,0,824
4,10,2018,20,C,0.0,0.0,0.0,0.0,0.0,0.0,8.9488,0,735
...,...,...,...,...,...,...,...,...,...,...,...,...,...
492868,1000008,2019,38,C,0.0,0.0,0.0,0.0,0.0,0.0,37992.9648,0,886
492869,1000008,2019,39,C,0.0,0.0,0.0,0.0,0.0,0.0,36192.8112,0,886
492870,1000008,2019,40,C,0.0,0.0,0.0,0.0,0.0,0.0,33515.8544,0,886
492871,1000008,2019,41,C,0.0,0.0,0.0,0.0,0.0,0.0,33598.4768,0,886


In [118]:
linear1=pd.get_dummies(linear1, columns=['EmployeeType'])
linear1=pd.get_dummies(linear1, columns=['Week'])
linear1=pd.get_dummies(linear1, columns=['Year'])

for col in linear1.columns:
    print(col)

USER_ID
CourseType_Corporate_Self
CourseType_Product_Self
CourseType_Theme_Self
CourseType_Corporate_Mandatory
CourseType_Product_Mandatory
CourseType_Theme_Mandatory
Sales
Trained
HiringDate_HR
EmployeeType_C
EmployeeType_P_FT
EmployeeType_P_PT
Week_1
Week_2
Week_3
Week_4
Week_5
Week_6
Week_7
Week_8
Week_9
Week_10
Week_11
Week_12
Week_13
Week_14
Week_15
Week_16
Week_17
Week_18
Week_19
Week_20
Week_21
Week_22
Week_23
Week_24
Week_25
Week_26
Week_27
Week_28
Week_29
Week_30
Week_31
Week_32
Week_33
Week_34
Week_35
Week_36
Week_37
Week_38
Week_39
Week_40
Week_41
Week_42
Week_43
Week_44
Week_45
Week_46
Week_47
Week_48
Week_49
Week_50
Week_51
Week_52
Year_2018
Year_2019


In [123]:
linear1['SUMTRAIN'] = linear1.loc[:, 'CourseType_Corporate_Self':'CourseType_Theme_Mandatory'].sum(1)
linear1['diff'] = linear1.groupby('USER_ID')['SUMTRAIN'].diff()
linear11 = linear1

In [126]:
linear11['diff']= linear11.apply(nan_with_cond , axis=1)

In [128]:
linear11['TRAIN2'] = linear11.apply(per2, axis=1)

In [132]:
linear11['Sales_ln'] = np.log(linear11['Sales'])
linear11['SUMTRAIN_ln'] = np.log(linear11['SUMTRAIN']+5)
linear11['HiringDate_HR_ln'] = np.log(linear11['HiringDate_HR'])

columns=['CourseType_Corporate_Self','CourseType_Product_Self','CourseType_Theme_Self','CourseType_Corporate_Mandatory','CourseType_Product_Mandatory','CourseType_Theme_Mandatory']

for i in columns:
    linear11[i+'_ln'] = np.log(linear11[i]+5)

In [140]:
linear1

,USER_ID,CourseType_Corporate_Self,CourseType_Product_Self,CourseType_Theme_Self,CourseType_Corporate_Mandatory,CourseType_Product_Mandatory,CourseType_Theme_Mandatory,Sales,Trained,HiringDate_HR,...,TRAIN2,Sales_ln,SUMTRAIN_ln,HiringDate_HR_ln,CourseType_Corporate_Self_ln,CourseType_Product_Self_ln,CourseType_Theme_Self_ln,CourseType_Corporate_Mandatory_ln,CourseType_Product_Mandatory_ln,CourseType_Theme_Mandatory_ln
0,9,0.0,0.0,0.0,0.0,0.0,0.0,6.2608,0,824,...,0,1.834308,1.609438,6.714171,1.609438,1.609438,1.609438,1.609438,1.609438,1.609438
1,9,0.0,0.0,0.0,0.0,0.0,0.0,21.2576,0,824,...,0,3.056714,1.609438,6.714171,1.609438,1.609438,1.609438,1.609438,1.609438,1.609438
2,9,0.0,0.0,0.0,0.0,0.0,0.0,10.0688,0,824,...,0,2.309442,1.609438,6.714171,1.609438,1.609438,1.609438,1.609438,1.609438,1.609438
3,9,0.0,0.0,0.0,0.0,0.0,0.0,6.7088,0,824,...,0,1.903420,1.609438,6.714171,1.609438,1.609438,1.609438,1.609438,1.609438,1.609438
4,10,0.0,0.0,0.0,0.0,0.0,0.0,8.9488,0,735,...,0,2.191519,1.609438,6.599870,1.609438,1.609438,1.609438,1.609438,1.609438,1.609438
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492868,1000008,0.0,0.0,0.0,0.0,0.0,0.0,37992.9648,0,886,...,0,10.545156,1.609438,6.786717,1.609438,1.609438,1.609438,1.609438,1.609438,1.609438
492869,1000008,0.0,0.0,0.0,0.0,0.0,0.0,36192.8112,0,886,...,0,10.496616,1.609438,6.786717,1.609438,1.609438,1.609438,1.609438,1.609438,1.609438
492870,1000008,0.0,0.0,0.0,0.0,0.0,0.0,33515.8544,0,886,...,0,10.419774,1.609438,6.786717,1.609438,1.609438,1.609438,1.609438,1.609438,1.609438
492871,1000008,0.0,0.0,0.0,0.0,0.0,0.0,33598.4768,0,886,...,0,10.422236,1.609438,6.786717,1.609438,1.609438,1.609438,1.609438,1.609438,1.609438


In [144]:
model1 = ols('Sales_ln ~ CourseType_Corporate_Self_ln + CourseType_Product_Self_ln + CourseType_Theme_Self_ln +\
                            CourseType_Corporate_Mandatory_ln + CourseType_Product_Mandatory_ln +\
                            CourseType_Theme_Mandatory_ln + TRAIN2 + HiringDate_HR_ln + EmployeeType_C + EmployeeType_P_FT +Week_2+Week_3+\
                            Week_4+Week_5+Week_6+Week_7+Week_8+Week_9+Week_10+Week_11+Week_12+Week_13+Week_14+Week_15+Week_16+Week_17+\
                            Week_18+Week_19+Week_20+Week_21+Week_22+Week_23+Week_24+Week_25+Week_26+Week_27+Week_28+Week_29+Week_30+Week_31+\
                            Week_32+Week_33+Week_34+Week_35+Week_36+Week_37+Week_38+Week_39+Week_40+Week_41+Week_42+Week_43+Week_44+\
                            Week_45+Week_46+Week_47+Week_48+Week_49+Week_50+Week_51+Week_52'\
             , data=linear11).fit()

print(model1.summary())

                            OLS Regression Results                            
Dep. Variable:               Sales_ln   R-squared:                       0.152
Model:                            OLS   Adj. R-squared:                  0.152
Method:                 Least Squares   F-statistic:                     1453.
Date:                Tue, 05 May 2020   Prob (F-statistic):               0.00
Time:                        22:17:19   Log-Likelihood:            -6.1438e+05
No. Observations:              492873   AIC:                         1.229e+06
Df Residuals:                  492811   BIC:                         1.230e+06
Df Model:                          61                                         
Covariance Type:            nonrobust                                         
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Interc

# OVERALL

In [149]:
w = {'CourseType_Corporate_Self':['max'], 'CourseType_Product_Self':['max'], 'CourseType_Theme_Self':['max'],'CourseType_Corporate_Mandatory':['max'],\
    'CourseType_Product_Mandatory':['max'],'CourseType_Theme_Mandatory':['max'],'Sales':['sum'],'Retail Units':['sum'],'Trained':['max'],'HiringDate_HR':['max']}

overall= merge3.groupby(['USER_ID','Year','Week','EmployeeType']).agg(w)

overall

CourseType_Corporate_Self  \
                                                     max   
USER_ID Year Week EmployeeType                             
9       2018 7    C                                  0.0   
             9    C                                  0.0   
             10   C                                  0.0   
             11   C                                  0.0   
             16   C                                  0.0   
...                                                  ...   
1000008 2019 38   C                                  0.0   
             39   C                                  0.0   
             40   C                                  0.0   
             41   C                                  0.0   
             42   C                                  0.0   

                               CourseType_Product_Self CourseType_Theme_Self  \
                                                   max                   max   
USER_ID Year Week EmployeeType                                                 
9       2018 7    C                                0.0                   0.0   
             9    C                                0.0                   0.0   
             10   C                                0.0                   0.0   
             11   C                                0.0                   0.0   
             16   C                                0.0                   0.0   
...                                                ...                   ...   
1000008 2019 38   C                                0.0                   0.0   
             39   C                                0.0                   0.0   
             40   C                                0.0                   0.0   
             41   C                                0.0                   0.0   
             42   C                                0.0                   0.0   

                               CourseType_Corporate_Mandatory  \
                                                          max   
USER_ID Year Week EmployeeType                                  
9       2018 7    C                                       0.0   
             9    C                                       0.0   
             10   C                                       0.0   
             11   C                                       0.0   
             16   C                                       0.0   
...                                                       ...   
1000008 2019 38   C                                       0.0   
             39   C                                       0.0   
             40   C                                       0.0   
             41   C                                       0.0   
             42   C                                       0.0   

                               CourseType_Product_Mandatory  \
                                                        max   
USER_ID Year Week EmployeeType                                
9       2018 7    C                                     0.0   
             9    C                                     0.0   
             10   C                                     0.0   
             11   C                                     0.0   
             16   C                                     0.0   
...                                                     ...   
1000008 2019 38   C                                     0.0   
             39   C                                     0.0   
             40   C                                     0.0   
             41   C                                     0.0   
             42   C                                     0.0   

                               CourseType_Theme_Mandatory       Sales  \
                                                      max         sum   
USER_ID Year Week EmployeeType                                          
9       2018 7    C                                   0.

In [150]:
overall = overall.reset_index()
overall

,USER_ID,Year,Week,EmployeeType,CourseType_Corporate_Self,CourseType_Product_Self,CourseType_Theme_Self,CourseType_Corporate_Mandatory,CourseType_Product_Mandatory,CourseType_Theme_Mandatory,Sales,Retail Units,Trained,HiringDate_HR
,,,,,max,max,max,max,max,max,sum,sum,max,max
0,9,2018,7,C,0.0,0.0,0.0,0.0,0.0,0.0,6.2608,0.0,0,824
1,9,2018,9,C,0.0,0.0,0.0,0.0,0.0,0.0,21.2576,0.0,0,824
2,9,2018,10,C,0.0,0.0,0.0,0.0,0.0,0.0,10.0688,0.0,0,824
3,9,2018,11,C,0.0,0.0,0.0,0.0,0.0,0.0,6.7088,0.0,0,824
4,9,2018,16,C,0.0,0.0,0.0,0.0,0.0,0.0,160.1600,1.0,0,824
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
729173,1000008,2019,38,C,0.0,0.0,0.0,0.0,0.0,0.0,73734.1696,266.0,0,886
729174,1000008,2019,39,C,0.0,0.0,0.0,0.0,0.0,0.0,76079.7296,264.0,0,886
729175,1000008,2019,40,C,0.0,0.0,0.0,0.0,0.0,0.0,76182.7920,269.0,0,886


In [151]:
overall.columns = ['USER_ID','Year','Week','EmployeeType','CourseType_Corporate_Self','CourseType_Product_Self',\
                   'CourseType_Theme_Self','CourseType_Corporate_Mandatory','CourseType_Product_Mandatory','CourseType_Theme_Mandatory',\
                   'Sales','Retail_Units','Trained','HiringDate_HR']
overall

,USER_ID,Year,Week,EmployeeType,CourseType_Corporate_Self,CourseType_Product_Self,CourseType_Theme_Self,CourseType_Corporate_Mandatory,CourseType_Product_Mandatory,CourseType_Theme_Mandatory,Sales,Retail_Units,Trained,HiringDate_HR
0,9,2018,7,C,0.0,0.0,0.0,0.0,0.0,0.0,6.2608,0.0,0,824
1,9,2018,9,C,0.0,0.0,0.0,0.0,0.0,0.0,21.2576,0.0,0,824
2,9,2018,10,C,0.0,0.0,0.0,0.0,0.0,0.0,10.0688,0.0,0,824
3,9,2018,11,C,0.0,0.0,0.0,0.0,0.0,0.0,6.7088,0.0,0,824
4,9,2018,16,C,0.0,0.0,0.0,0.0,0.0,0.0,160.1600,1.0,0,824
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
729173,1000008,2019,38,C,0.0,0.0,0.0,0.0,0.0,0.0,73734.1696,266.0,0,886
729174,1000008,2019,39,C,0.0,0.0,0.0,0.0,0.0,0.0,76079.7296,264.0,0,886
729175,1000008,2019,40,C,0.0,0.0,0.0,0.0,0.0,0.0,76182.7920,269.0,0,886
729176,1000008,2019,41,C,0.0,0.0,0.0,0.0,0.0,0.0,75567.7104,247.0,0,886


In [152]:
overall=pd.get_dummies(overall, columns=['EmployeeType'])
overall=pd.get_dummies(overall, columns=['Week'])
overall=pd.get_dummies(overall, columns=['Year'])

overall

,USER_ID,CourseType_Corporate_Self,CourseType_Product_Self,CourseType_Theme_Self,CourseType_Corporate_Mandatory,CourseType_Product_Mandatory,CourseType_Theme_Mandatory,Sales,Retail_Units,Trained,...,Week_45,Week_46,Week_47,Week_48,Week_49,Week_50,Week_51,Week_52,Year_2018,Year_2019
0,9,0.0,0.0,0.0,0.0,0.0,0.0,6.2608,0.0,0,...,0,0,0,0,0,0,0,0,1,0
1,9,0.0,0.0,0.0,0.0,0.0,0.0,21.2576,0.0,0,...,0,0,0,0,0,0,0,0,1,0
2,9,0.0,0.0,0.0,0.0,0.0,0.0,10.0688,0.0,0,...,0,0,0,0,0,0,0,0,1,0
3,9,0.0,0.0,0.0,0.0,0.0,0.0,6.7088,0.0,0,...,0,0,0,0,0,0,0,0,1,0
4,9,0.0,0.0,0.0,0.0,0.0,0.0,160.1600,1.0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
729173,1000008,0.0,0.0,0.0,0.0,0.0,0.0,73734.1696,266.0,0,...,0,0,0,0,0,0,0,0,0,1
729174,1000008,0.0,0.0,0.0,0.0,0.0,0.0,76079.7296,264.0,0,...,0,0,0,0,0,0,0,0,0,1
729175,1000008,0.0,0.0,0.0,0.0,0.0,0.0,76182.7920,269.0,0,...,0,0,0,0,0,0,0,0,0,1
729176,1000008,0.0,0.0,0.0,0.0,0.0,0.0,75567.7104,247.0,0,...,0,0,0,0,0,0,0,0,0,1


In [153]:
overall['SUMTRAIN'] = overall.loc[:, 'CourseType_Corporate_Self':'CourseType_Theme_Mandatory'].sum(1)

In [154]:
overall['diff'] = overall.groupby('USER_ID')['SUMTRAIN'].diff()

In [156]:
overall['diff']= overall.apply(nan_with_cond , axis=1)

In [157]:
overall

,USER_ID,CourseType_Corporate_Self,CourseType_Product_Self,CourseType_Theme_Self,CourseType_Corporate_Mandatory,CourseType_Product_Mandatory,CourseType_Theme_Mandatory,Sales,Retail_Units,Trained,...,Week_47,Week_48,Week_49,Week_50,Week_51,Week_52,Year_2018,Year_2019,SUMTRAIN,diff
0,9,0.0,0.0,0.0,0.0,0.0,0.0,6.2608,0.0,0,...,0,0,0,0,0,0,1,0,0.0,0.0
1,9,0.0,0.0,0.0,0.0,0.0,0.0,21.2576,0.0,0,...,0,0,0,0,0,0,1,0,0.0,0.0
2,9,0.0,0.0,0.0,0.0,0.0,0.0,10.0688,0.0,0,...,0,0,0,0,0,0,1,0,0.0,0.0
3,9,0.0,0.0,0.0,0.0,0.0,0.0,6.7088,0.0,0,...,0,0,0,0,0,0,1,0,0.0,0.0
4,9,0.0,0.0,0.0,0.0,0.0,0.0,160.1600,1.0,0,...,0,0,0,0,0,0,1,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
729173,1000008,0.0,0.0,0.0,0.0,0.0,0.0,73734.1696,266.0,0,...,0,0,0,0,0,0,0,1,0.0,0.0
729174,1000008,0.0,0.0,0.0,0.0,0.0,0.0,76079.7296,264.0,0,...,0,0,0,0,0,0,0,1,0.0,0.0
729175,1000008,0.0,0.0,0.0,0.0,0.0,0.0,76182.7920,269.0,0,...,0,0,0,0,0,0,0,1,0.0,0.0
729176,1000008,0.0,0.0,0.0,0.0,0.0,0.0,75567.7104,247.0,0,...,0,0,0,0,0,0,0,1,0.0,0.0


In [158]:
overall['Sales_ln'] = np.log(overall['Sales'])
overall['SUMTRAIN_ln'] = np.log(overall['SUMTRAIN']+5)
overall['HiringDate_HR_ln'] = np.log(overall['HiringDate_HR'])

columns=['CourseType_Corporate_Self','CourseType_Product_Self','CourseType_Theme_Self','CourseType_Corporate_Mandatory','CourseType_Product_Mandatory','CourseType_Theme_Mandatory']

for i in columns:
    overall[i+'_ln'] = np.log(overall[i]+5)

In [159]:
overall['TRAIN2'] = overall.apply(per2, axis=1)

In [163]:
model3 = ols('Sales_ln ~ CourseType_Corporate_Self_ln + CourseType_Product_Self_ln + CourseType_Theme_Self_ln +\
                            CourseType_Corporate_Mandatory_ln + CourseType_Product_Mandatory_ln +\
                            CourseType_Theme_Mandatory_ln + TRAIN2 + HiringDate_HR_ln + EmployeeType_C + EmployeeType_P_FT +Week_2+Week_3+\
                            Week_4+Week_5+Week_6+Week_7+Week_8+Week_9+Week_10+Week_11+Week_12+Week_13+Week_14+Week_15+Week_16+Week_17+\
                            Week_18+Week_19+Week_20+Week_21+Week_22+Week_23+Week_24+Week_25+Week_26+Week_27+Week_28+Week_29+Week_30+Week_31+\
                            Week_32+Week_33+Week_34+Week_35+Week_36+Week_37+Week_38+Week_39+Week_40+Week_41+Week_42+Week_43+Week_44+\
                            Week_45+Week_46+Week_47+Week_48+Week_49+Week_50+Week_51+Week_52+Year_2019'\
             , data=overall).fit()

print(model3.summary())

                            OLS Regression Results                            
Dep. Variable:               Sales_ln   R-squared:                       0.242
Model:                            OLS   Adj. R-squared:                  0.242
Method:                 Least Squares   F-statistic:                     3746.
Date:                Tue, 05 May 2020   Prob (F-statistic):               0.00
Time:                        22:41:21   Log-Likelihood:            -1.0317e+06
No. Observations:              729178   AIC:                         2.064e+06
Df Residuals:                  729115   BIC:                         2.064e+06
Df Model:                          62                                         
Covariance Type:            nonrobust                                         
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Interc